In [1]:
# if you want to have a pop up sign in method instead please uncomment  on ee.Authenticate and ee.Initialize, but comment on below

import ee

# Trigger the authentication flow. if you want to user json, please comment this
ee.Authenticate()
# Initialize the library.
ee.Initialize()

# here we try to use json api information instead (service account) - comment below if you use above methode
#import ee
import math
from datetime import datetime

# uncomment this if you want to use json instead
#service_account = 'iqbalpythonapi@bukit30project.iam.gserviceaccount.com'
#credentials = ee.ServiceAccountCredentials(service_account, './bukit30project-4d92e5b46ea7.json')
#ee.Initialize(credentials)

Enter verification code: 4/1AZEOvhXj29hfQslaT5eFKfpKFG_9OPcthsJYq5y7IZpdob3JC7gEyHhsRWY

Successfully saved authorization token.


In [3]:
import geemap#.foliumap as geemap
import geopandas as gpd
import numpy as np
from sklearn.metrics import confusion_matrix

#Algorith outcome input (Forest Basemap (High baseline))
algor_process = 'fcd_galcit'  #look below
# Enter-input 'Sentinel' for prioritizing 'Sentinel' to get have Forest (High Baseline) and fill in the 10 years rule by Hansen,
# Enter-input 'Hansen' for prioritizing TCL based forest (High Baseline), and 10 years rule and fill in by Sentinel for the uncertain areas (No Changes in areas that Crown cover)
# Enter-input 'arief_hansen' for using empirical equation and get 10 years rule info with hansen after,
# Enter-input 'hansen_arief' for prioritizing TCL based forest Hansen, and fill in by arief data in missing info
# Enter-input 'fcd_galcit' for using fcd method and thermal index of landsat

eval_process = False

calc_areas = False

exporting = True

##### INPUT ###########################
title_map = 'AOI_SRA_LANDSCAPE'

#AOI File (SHP) - added shp_local: please make a folder in the main directory: shp_local and put here as an input (example) - shp local not publish to github
AOIt_shp = './shp_local/AOI_SRA_LANDSCAPE.shp'
AOIt_shp_plot = geemap.shp_to_ee(AOIt_shp)
crs_input = 'EPSG:4326'

# Edit this to False if there is no smaller AOI to overlay with
small_aoi = True

if small_aoi:
    AOIsmall_shp = './shp_local/SRA_Concession.shp'
    AOIsmall = geemap.shp_to_ee(AOIsmall_shp)
    
# edit this to False if there is no third layer smaller AOI to overlay with again
smaller_aoi = True
if smaller_aoi:
    AOIsmaller_shp = './shp_local/Deliniate_FIX_GOZONE_new_plot.shp'
    AOIsmaller = geemap.shp_to_ee(AOIsmaller_shp)

################
#for area id in shapefile that identified the data, and will converted into raster
OID = 'id'  #IMPORTANT TO CHECK OID based on the column ID
#############################################

AOI = AOIt_shp_plot

# Accuracy point Reference
if eval_process:
    ref_shp = './shp_local/ref_point_B30.shp' #### CHANGE THIS AS PER AOI LOCATION
    #reference point input to gee object and geopandas
    ref_point_loc = geemap.shp_to_ee(ref_shp)
    gdf_ref = gpd.read_file(ref_shp)
    #print(gdf_ref.head())

######## HANSEN ##################
# Hansen 10 years rule and Forest - Tree Cover Hansen
#define tree cover minimum that classified as forest e.g., Indonesia > 30%, hence 30
tree_cover_forest = 30 
pixel_number = 3 #define minimum mapping unit classified as forest, 1 pixel for landsat 30mx30, 3 pixel is forest ~> 0.27 Ha
year_start_loss = 20 #define start year to track as 10 years rule (e.g., 2012 to 2022 (track 10 years rule), hence 12. format= 00->2000, 12->2012)

#CloudLess Images Sentinel
START_DATE = '2022-1-1'   #Change this as perusal
END_DATE = '2022-12-31'

#Make an image out of the AOI area attribute -> convert featurecollection into raster (image) for overlaying tools
AOI_img = AOI.filter(ee.Filter.notNull([OID])).reduceToImage(
    properties= [OID],
    reducer= ee.Reducer.first()
)
#Map.addLayer(AOI_img) #check if it works
################

#FOR FOREST DETECTION - Sentinel

# Colors fo raster symbology
water = '#3380cc' #Blue 
hi_forest = '#006837' #Strong green
low_forest = '#3ea540' #Medium green
grass_land = '#baf096' #Light green
bare_land = '#ad8855' #Brown
other = '#000000'

#COEFICIENTS: These coeficients are orientative
#and some tweak may be needed depending on the
#location and case of study

#NDWI water limit
ndwi_hi_sentinel = 0.05 # for Sentinel
ndwi_hi_landsat = 0.1 # for landsat
ndwi_hi_planet = -0.2 # for Planet Labs

#Bare soil index (BI), soil limit
bi_hi = 2

#NDVI high and low limits
ndvi_lo = 0.25     #0.20 for L1C (suggested value)
                   #0.25 for L2A (suggested value)
ndvi_hi = 0.45     #0.40 for L1C (suggested value)
                   #0.45 for L2A (suggested value)

#Shadow index (SI) high and low limits
si_lo = 0.92       #0.90 for L1C (suggested value) 
                   #0.92 for L2A (suggested value)
si_hi = 0.95       #0.93 for L1C (suggested value) 
                   #0.95 for L2A (suggested value)
#############################

# output Band Names
band_name_image = 'Class'


# Threshold y (High Baseline ton) - arief et al
y_threshold = 90

# DEM
type_DEM = 'srtm'

alos = ee.Image('JAXA/ALOS/AW3D30_V1_1')
srtm = ee.Image('USGS/SRTMGL1_003')

# input scale based on the image property
#pca_scale = image_corrected.projection().nominalScale()
pca_scale = 60


# Decide to use Thermal Index or not - if the result is poor because the availability of image for TI (Thermal Index noise) 
# please set to False
TI_include = True

## FCD Threshold
high_forest = 70
yrf_forest = 50
shrub_grass = 30

#revision of FCD threshold if TI is not included
if TI_include == False:
    high_forest = high_forest
    yrf_forest = yrf_forest
    shrub_grass = shrub_grass


##################################################################################

In [4]:
l4 = ee.ImageCollection("LANDSAT/LT04/C02/T1_L2")
l7 = ee.ImageCollection("LANDSAT/LE07/C02/T1_L2")
l9 = ee.ImageCollection("LANDSAT/LC09/C02/T1_L2")
l8 = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")
l5 = ee.ImageCollection("LANDSAT/LT05/C02/T1_L2")

# need to have this threshold for a map function of thermal index, to ensure the data available,
#  if error 400 in TI or SSI data creation, please reduce the cloud cover
cloud_cover_threshold = 50
l4_raw = ee.ImageCollection("LANDSAT/LT04/C02/T1")
l7_raw = ee.ImageCollection("LANDSAT/LE07/C02/T1")
l9_raw = ee.ImageCollection("LANDSAT/LC09/C02/T1")
l8_raw = ee.ImageCollection("LANDSAT/LC08/C02/T1")
l5_raw = ee.ImageCollection("LANDSAT/LT05/C02/T1")

In [5]:
PlanetNICFI = ee.ImageCollection("projects/planet-nicfi/assets/basemaps/asia")
filteredNicfi = PlanetNICFI.filterDate(START_DATE, END_DATE) \
                     .filterBounds(AOI)

def scaling_planet(image):
    sr_image = image.select(['R','G','B','N'], ['red','green', 'blue','nir']).multiply(0.0001)
    return sr_image

filtered_scaled = filteredNicfi.map(scaling_planet).median().clip(AOI)

In [6]:
# additional conditioning input:
if algor_process == 'fcd_galcit' and TI_include:
    I_satellite = 'LANDSAT'
elif algor_process == 'fcd_galcit' and TI_include == False:
    print('choose Planet NICFI')
    I_satellite = 'PlanetNicfi'
else:
    I_satellite = 'Sentinel'
print(I_satellite)

LANDSAT


In [7]:
############################ Deforestation areas - TCL
# hansen - updated version - global data
gfc = ee.Image("UMD/hansen/global_forest_change_2022_v1_10")

#Canopy cover percentage (e.g. 30%), for Indonesia
cc = ee.Number(tree_cover_forest)

#Minimum forest area in pixels (e.g. 3 pixels, ~ 0.27 ha in this example).
pixels = ee.Number(pixel_number)

#Minimum mapping area for tree loss (usually same as the minimum forest area).
lossPixels = pixels

canopyCover = gfc.select(['treecover2000'])
canopyCoverThreshold = canopyCover.gte(cc).selfMask()

#Use connectedPixelCount() to get contiguous area.
contArea = canopyCoverThreshold.connectedPixelCount()
#Apply the minimum area requirement.
minArea = contArea.gte(pixels).selfMask()

prj = gfc.projection()
scale = prj.nominalScale()
ForestArea2000Hansen = minArea.reproject(prj.atScale(scale))

Map = geemap.Map(center=(-3, 115), zoom=4)
Map.centerObject(AOI, 10)
Map.addLayer(ForestArea2000Hansen, {
    'palette': ['#96ED89']
}, 'tree cover: >= min canopy cover & area (light green)')

#create visual boundary color only
empty = ee.Image().byte()
AOIm = empty.paint(AOI,0,10)

#Map.addLayer(AOIm,{'palette': '#e043f3'},'AOI') #still buggy
#Map.addLayer(AOIm,{},'AOI')

'''
forestArea = minArea.multiply(ee.Image.pixelArea()).divide(10000)
forestSize = forestArea.reduceRegion(
    reducer=ee.Reducer.sum(),
    geometry=AOI.geometry(),
    scale=30,
    maxPixels=1e13
)

print(
    'Year 2000 tree cover (ha) \nmeeting minimum canopy cover and \nforest area thresholds \n ',
    forestSize.get('treecover2000'))
    
'''

'''
pixelCount = minArea.reduceRegion(
    reducer=ee.Reducer.count(),
    geometry=AOI.geometry(),
    scale=30,
    maxPixels=1e13
)
onePixel = forestSize.getNumber('treecover2000').divide(pixelCount.getNumber('treecover2000'))
minAreaUsed = onePixel.multiply(pixels)
print('Minimum forest area used (ha)\n ', minAreaUsed)
'''

"\npixelCount = minArea.reduceRegion(\n    reducer=ee.Reducer.count(),\n    geometry=AOI.geometry(),\n    scale=30,\n    maxPixels=1e13\n)\nonePixel = forestSize.getNumber('treecover2000').divide(pixelCount.getNumber('treecover2000'))\nminAreaUsed = onePixel.multiply(pixels)\nprint('Minimum forest area used (ha)\n ', minAreaUsed)\n"

In [8]:
##### Visualized LANDSAT CLOUDLESS FROM HANSEN DATASET LAST YEAR
LastImageLandsat = gfc.select(['last_b30', 'last_b40', 'last_b50', 'last_b70']).rename(['red', 'nir', 'swir1', 'swir2']).clip(AOI)
Map.addLayer(LastImageLandsat, {'bands': ['swir2', 'nir', 'red'], 'min': 0, 'max': 600, 'gamma': 1.5 },
                f'Landsat Hansen Last Year {END_DATE}')

In [9]:
## FOR LANDSAT

def filter_col(col, AOI, date):
    return col.filterDate(date[0], date[1]).filterBounds(AOI).filter(ee.Filter.lte('CLOUD_COVER', cloud_cover_threshold))

def cloud_mask_tm(image, sr=True):
    qa = image.select('QA_PIXEL')
    dilated = 1 << 1
    cloud = 1 << 3
    shadow = 1 << 4
    mask = qa.bitwiseAnd(dilated).eq(0) \
        .And(qa.bitwiseAnd(cloud).eq(0)) \
        .And(qa.bitwiseAnd(shadow).eq(0))
  
    if sr:
        optical_bands = image.select(['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B7'], ['blue', 'green', 'red', 'nir', 'swir1', 'swir2']) \
                                            .updateMask(mask) \
                                            .multiply(0.0000275).add(-0.2)
        
        
        thermal_band = image.select(['ST_B6'], ['thermal']).updateMask(mask) \
                                            .multiply(0.00341802).add(149.0)
        
        
    else:
        optical_bands = image.select(['B1', 'B2', 'B3', 'B4', 'B5', 'B7'], ['blue', 'green', 'red', 'nir', 'swir1', 'swir2']) \
                                            .updateMask(mask) \
                                            .multiply(0.0001)
        
        thermal_band = image.select(['B6'], ['thermal']).updateMask(mask) \
                                            .multiply(0.0001)
    
    cloudM = mask.rename('cloudM')
    
    imageCloudMasked = image.addBands(optical_bands, None, True) \
                            .addBands(thermal_band, None, True)
    
    imageCloudMasked = imageCloudMasked.addBands(cloudM)
    
    return imageCloudMasked

def cloud_mask_oli(image, sr=True):
    qa = image.select('QA_PIXEL')
    dilated = 1 << 1
    cirrus = 1 << 2
    cloud = 1 << 3
    shadow = 1 << 4
    mask = qa.bitwiseAnd(dilated).eq(0) \
        .And(qa.bitwiseAnd(cirrus).eq(0)) \
        .And(qa.bitwiseAnd(cloud).eq(0)) \
        .And(qa.bitwiseAnd(shadow).eq(0))

    if sr:
        #print('sr=True')
        optical_bands = image.select(['SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7'], ['blue', 'green', 'red', 'nir', 'swir1', 'swir2']) \
                        .updateMask(mask) \
                        .multiply(0.0000275).add(-0.2)

        thermal_band = image.select(['ST_B10'], ['thermal']).updateMask(mask) \
                        .multiply(0.00341802).add(149.0)
        
    else:
        optical_bands = image.select(['B2', 'B3', 'B4', 'B5', 'B6', 'B7'], ['blue', 'green', 'red', 'nir', 'swir1', 'swir2']) \
                        .updateMask(mask) #\
                        #.multiply(0.0001)

        # image = detect_cloud_shadow(image)
        
        thermal_band = image.select(['B10'], ['thermal']) \
                        .updateMask(mask) #\
                        #.multiply(0.0001)
            
        #best_image_year = detect_cloud_shadow(best_image_year)
        
        AlB10 = ee.Number(image.get('RADIANCE_ADD_BAND_10'))
        M1B10 = ee.Number(image.get('RADIANCE_MULT_BAND_10'))
        
        radiance = thermal_band.multiply(M1B10).add(AlB10)
        
        K_1 = ee.Number(image.get('K1_CONSTANT_BAND_10'))
        K_2 = ee.Number(image.get('K2_CONSTANT_BAND_10'))
        A = ee.Number(image.get('RADIANCE_ADD_BAND_10'))
        M = ee.Number(image.get('RADIANCE_MULT_BAND_10'))

        # Getting the Thermal Index - source??
        TI = radiance.expression('(K_2) / log((K_1 / TEMP)+1)',{
              'K_2': K_2,
              'K_1': K_1,
              'TEMP':radiance
            }
          ).rename('TI')
        
    
    cloudM = mask.rename('cloudM')
    
    imageCloudMasked = image.addBands(optical_bands, None, True) \
                            .addBands(thermal_band, None, True)
    if sr==False:
        #print('sr=False')
        imageCloudMasked = imageCloudMasked.addBands(TI, None, True)
    
    imageCloudMasked = imageCloudMasked.addBands(cloudM)   
    
    return imageCloudMasked

# Composite function
def landsat457(roi, date):
    collection = filter_col(l4, roi, date).merge(filter_col(l5, roi, date))
    image_collection = collection.map(cloud_mask_tm)#.median().clip(roi)
    return image_collection

def landsat89(roi, date):
    collection = filter_col(l8, roi, date).merge(filter_col(l9, roi, date))
    image_collection = collection.map(cloud_mask_oli)#.median().clip(roi)
    return image_collection

def landsat_raw457(roi, date):
    collection = filter_col(l4_raw, roi, date).merge(filter_col(l5_raw, roi, date))
    image_collection = collection.map(lambda image: cloud_mask_tm(image, sr=False))#.median().clip(roi)
    return image_collection
    
def landsat_raw89(roi, date):
    collection = filter_col(l8_raw, roi, date).merge(filter_col(l9_raw, roi, date))
    image_collection = collection.map(lambda image: cloud_mask_oli(image, sr=False))#.median().clip(roi)
    return image_collection




In [10]:
# Topographic Correction Function ========================================================================================
# Define the function to convert degrees to radians
def radians(img):
    return img.toFloat().multiply(3.14159265359).divide(180)

# Define the function to calculate illumination parameters and angles
def calcIlluminationParametersAndAngles(image):
    extent = image.geometry().bounds(1)

    # Select DEM, calculate aspect and slope
    
    if type_DEM == 'alos':
        DEM = alos
    else: 
        DEM = srtm
    
    DEM = DEM.clip(extent)
    boxcar = ee.Kernel.square(radius=3, units='pixels', normalize=True)
    DEMs = DEM.convolve(boxcar)
    SLP_deg = ee.Terrain.slope(DEMs)
    SLP = radians(SLP_deg)
    ASP_deg = ee.Terrain.aspect(DEMs)
    ASP = radians(ASP_deg)
    cos_SLP = SLP.cos()

    if I_satellite == 'LANDSAT':
        AZ = ee.Number(image.get('SUN_AZIMUTH'))
        ZE = ee.Number(image.get('SUN_ELEVATION'))
    elif I_satellite == 'Sentinel':
        AZ = ee.Number(image.get('MEAN_SOLAR_AZIMUTH_ANGLE'))
        ZE = ee.Number(image.get('MEAN_SOLAR_ZENITH_ANGLE'))

    AZ_R = radians(ee.Image(AZ)).clip(extent)
    ZE_R = radians(ee.Image(ZE)).clip(extent)
    cos_ZE = ZE_R.cos()
    cos_ZE_SLP = cos_ZE.multiply(SLP.cos())
    cos_VA = ee.Image(0).clip(extent).cos()

    # Calculate cos(Z)⋅cos(s) + sin(Z)⋅sin(s)⋅cos(a - a')
    IL1 = AZ_R.subtract(ASP).cos().multiply(SLP.sin()).multiply(ZE_R.sin()).add(cos_ZE.multiply(SLP.cos()))
    IL2 = IL1.where(IL1.lte(0), 0)
    IL3 = IL2.where(IL2.gt(0), 1)
    IL = IL1.mask(IL3).select([0], ['IL'])

    # Create and return a dictionary
    keyList = ee.List(['IL', 'cos_ZE', 'cos_VA', 'cos_SLP', 'SLP', 'ASP'])
    valueList = ee.List([IL, cos_ZE, cos_VA, cos_SLP, SLP, ASP])
    dictionary = ee.Dictionary.fromLists(keyList, valueList)
    return dictionary

# Define the function to perform physical correction
def doPhysicalCorrection(imageUncorrected, ):
    parameters = calcIlluminationParametersAndAngles(imageUncorrected)

    cos_ZE = ee.Image(parameters.get('cos_ZE'))
    cos_VA = ee.Image(parameters.get('cos_VA'))
    cos_SLP = ee.Image(parameters.get('cos_SLP'))

    IL = ee.Image(parameters.get('IL'))

    imgTC = imageUncorrected.select().addBands(imageUncorrected.select(['blue', 'green', 'red', 'nir', 'swir1', 'swir2'])  \
                                        .mask(imageUncorrected.select(['blue', 'green', 'red', 'nir', 'swir1', 'swir2'])  \
                                        .And(IL).And(imageUncorrected.select('cloudM')))    \
                                        .multiply((((cos_ZE).add(cos_VA))).divide((IL.add(cos_SLP)))))   \
                                        .addBands(imageUncorrected.select('cloudM'))   \
                                        .set('system:time_start', imageUncorrected.get('system:time_start'))

    return ee.Image(imgTC)

In [11]:
# Spectral Index Formula for FCD
# scaling from old formula 8bit to modern images
def max_bands(image):
    maxRed = image.select('red').reduceRegion(reducer=ee.Reducer.max(),
                                                     geometry=AOI,
                                                     scale=pca_scale,
                                                     bestEffort=True,
                                                     tileScale=16)
    #print("MAXRED",maxRed)
    maxRed = ee.Dictionary(maxRed).toImage()
    maxGreen =image.select('green').reduceRegion(reducer=ee.Reducer.max(),
                                                 geometry=AOI,
                                                 scale=pca_scale,
                                                 bestEffort=True,
                                                 tileScale=16)
    maxGreen = ee.Dictionary(maxGreen).toImage()
    maxBlue = image.select('blue').reduceRegion(reducer=ee.Reducer.max(),
                                                geometry=AOI,
                                                scale=pca_scale,
                                                bestEffort=True,
                                                tileScale=16)
    maxBlue = ee.Dictionary(maxBlue).toImage()
    return [maxRed,maxGreen,maxBlue]

# Advanced Vegetation index  
def AVI_func(image):
    AVI = image.expression('((nir + 1 )* (maxRed-red) *(nir -red))**0.333',{
        'nir':image.select(['nir']),
        'red':image.select(['red']),
        'maxRed': max_bands(image)[0],
        }
    ).rename('AVI') 

    return AVI

# Bare Soil Index 
# https://www.researchgate.net/publication/319045433_Integration_of_GIS_and_Remote_Sensing_for_Evaluating_Forest_Canopy_Density_Index_in_Thai_Nguyen_Province_Vietnam/link/599293be0f7e9b433f415b40/download

def BSI_func(image):
    if I_satellite == 'LANDSAT':
        BSI = image.expression('(((SWIR1 + red) - (G + NIR)) / ((SWIR1 + red) + (G + NIR)))*100+100',{
            'SWIR1':image.select(['swir1']),
            'G':image.select(['green']),
            'red':image.select(['red']),
            'NIR':image.select(['nir']),
            }
        ).rename('BSI')
        
    elif I_satellite == 'PlanetNicfi':
        BSI = image.expression(
            '(NIR + GREEN + RED) / (NIR + GREEN - RED)',  {
                  'NIR': image.select('nir'),
                  'GREEN': image.select('green'),
                  'RED': image.select('red')
                    }
                ).rename('BSI')
  
    return BSI

def SI_func(image):
    SI = image.expression('((maxGreen-Green)*(maxBlue-Blue)*(maxRed-Red))**(1/3)',{  
        'Red':image.select(['red']),
        'Green':image.select(['green']), #//******
        'Blue':image.select(['blue']),
        'maxRed': max_bands(image)[0],
        'maxGreen': max_bands(image)[1],
        'maxBlue': max_bands(image)[2],
    }
    ).rename('SI')
    return SI



def normalization_100(image):
    #image_scale = ee.Number(image.projection().nominalScale())
    image_scale = pca_scale
    region = AOI
    def normalize_band(name):
        name = ee.String(name)
        band = image.select(name)
        
        mean_std = image.reduceRegion(
            reducer=ee.Reducer.mean().combine(ee.Reducer.stdDev(), None, True),
            geometry=region,
            scale=image_scale,
            maxPixels=10e9
        )
        
        mean = ee.Number(mean_std.get(name.cat('_mean')))
        std = ee.Number(mean_std.get(name.cat('_stdDev')))
        
        max_value = mean.add(std.multiply(3))
        min_value = mean.subtract(std.multiply(3))
        
        band1 = ee.Image(min_value).multiply(band.lt(min_value)).add(ee.Image(max_value).multiply(band.gt(max_value))) \
            .add(band.multiply(ee.Image(1).subtract(band.lt(min_value)).subtract(band.gt(max_value))))
        
        result_band = band1.subtract(min_value).divide(max_value.subtract(min_value))
        
        return result_band
    
    band_names = image.bandNames()
    unit_scale = ee.ImageCollection.fromImages(band_names.map(normalize_band)).toBands().rename(band_names)
    return unit_scale.multiply(100)

In [12]:
#### PRINCIPAL COMPONENT ANALYSIS ########
# Define the mean-centered function
def means_centered(image, region):
    bandNames = image.bandNames()

    # Mean center the data
    meanDict = image.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=region,
        scale=scale,
        maxPixels=1e9,
        tileScale=12,
        bestEffort=True,
    )
    means = ee.Image.constant(meanDict.values(bandNames))
    centered = image.subtract(means)

    return [centered, bandNames]

# Define the helper function to generate new band names
def getNewBandNames(prefix, bandNames):
    seq = ee.List.sequence(1, bandNames.length())
    return seq.map(lambda b: ee.String(prefix).cat(ee.Number(b).int().format()))

# Define the function to get principal components
def getPrincipalComponents(centered, scale, region, bandNames):
    # Collapse the bands into a 1D array per pixel
    arrays = centered.toArray()

    # Compute the covariance within the region
    covar = arrays.reduceRegion(
        reducer=ee.Reducer.centeredCovariance(),
        geometry=region,
        scale=scale,
        maxPixels=1e9,
        tileScale=16,
        bestEffort=True,
        
    )

    # Get the covariance array result and cast to an array
    covarArray = ee.Array(covar.get('array'))

    # Perform eigen analysis and slice apart the values and vectors
    eigens = covarArray.eigen()

    # Extract eigenvalues and eigenvectors
    eigenValues = eigens.slice(1, 0, 1)
    eigenVectors = eigens.slice(1, 1)

    # Convert the array image to 2D arrays
    arrayImage = arrays.toArray(1)

    # Left multiply the image array by the matrix of eigenvectors
    principalComponents = ee.Image(eigenVectors).matrixMultiply(arrayImage)

    # Square root of eigenvalues as a P-band image
    sdImage = ee.Image(eigenValues.sqrt()) \
        .arrayProject([0]).arrayFlatten([getNewBandNames('sd', bandNames)])

    # Normalize the principal components by their standard deviations
    return principalComponents \
        .arrayProject([0]) \
        .arrayFlatten([getNewBandNames('pc', bandNames)]) \
        .divide(sdImage)

In [13]:
### HISTORICAL DATA THRESHOLD - Get the TCL data LOSS PIXEL overall data from loss year in the input

treeLossYear = gfc.select(['lossyear'])
treeLoss = treeLossYear.gte(year_start_loss).selfMask() # tree loss in e.g., year > 2012 ####SHOULD CHANGE TO RECENT YEAR for the '12' number
#Select the tree loss within the derived tree cover
#(>= canopy cover and area requirements). THIS ONE ALREADY MASK TCL IN FOREST AREA
treecoverLoss = minArea.And(treeLoss).rename(f'lossfrom_{year_start_loss}').selfMask()

#Create connectedPixelCount() to get contiguous area.
contLoss = treecoverLoss.connectedPixelCount()
#Apply the minimum area requirement, and get the TCL data ---> minLoss - ACTUAL TCL AREA from Hansen since the year_start_loss
minLoss = contLoss.gte(lossPixels).selfMask()

'''
lossArea = minLoss.multiply(ee.Image.pixelArea()).divide(10000) #hectare
lossSize = lossArea.reduceRegion(
    reducer=ee.Reducer.sum(),
    geometry=AOI.geometry(),
    scale=30,
    maxPixels=1e13
)
print(
    '>Year 2012 tree loss (ha) \nmeeting minimum canopy cover and \nforest area thresholds \n ',
    lossSize.get(f'lossfrom_{year_start_loss}'))
'''


"\nlossArea = minLoss.multiply(ee.Image.pixelArea()).divide(10000) #hectare\nlossSize = lossArea.reduceRegion(\n    reducer=ee.Reducer.sum(),\n    geometry=AOI.geometry(),\n    scale=30,\n    maxPixels=1e13\n)\nprint(\n    '>Year 2012 tree loss (ha) \nmeeting minimum canopy cover and \nforest area thresholds \n ',\n    lossSize.get(f'lossfrom_{year_start_loss}'))\n"

In [14]:
##### START SENTINEL CLOUDLESS COMPOSITE AND MOSAICKING

#CLOUD_FILTER = 80
CLOUD_FILTER = 60
CLD_PRB_THRESH = 50
NIR_DRK_THRESH = 0.15
CLD_PRJ_DIST = 1
BUFFER = 50

def get_s2_sr_cld_col(aoi, start_date, end_date):
    # Import and filter S2 SR.
    s2_sr_col = (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
        .filterBounds(aoi)
        .filterDate(start_date, end_date)
        .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', CLOUD_FILTER)))

    # Import and filter s2cloudless.
    s2_cloudless_col = (ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY')
        .filterBounds(aoi)
        .filterDate(start_date, end_date))

    # Join the filtered s2cloudless collection to the SR collection by the 'system:index' property.
    return ee.ImageCollection(ee.Join.saveFirst('s2cloudless').apply(**{
        'primary': s2_sr_col,
        'secondary': s2_cloudless_col,
        'condition': ee.Filter.equals(**{
            'leftField': 'system:index',
            'rightField': 'system:index'
        })
    }))

#s2_sr_cld_col_eval = get_s2_sr_cld_col(AOI, START_DATE, END_DATE)

def add_cloud_bands(img):
    # Get s2cloudless image, subset the probability band.
    cld_prb = ee.Image(img.get('s2cloudless')).select('probability')

    # Condition s2cloudless by the probability threshold value.
    is_cloud = cld_prb.gt(CLD_PRB_THRESH).rename('clouds')

    # Add the cloud probability layer and cloud mask as image bands.
    return img.addBands(ee.Image([cld_prb, is_cloud]))

def add_shadow_bands(img):
    # Identify water pixels from the SCL band.
    not_water = img.select('SCL').neq(6)

    # Identify dark NIR pixels that are not water (potential cloud shadow pixels).
    SR_BAND_SCALE = 1e4
    dark_pixels = img.select('B8').lt(NIR_DRK_THRESH*SR_BAND_SCALE).multiply(not_water).rename('dark_pixels')

    # Determine the direction to project cloud shadow from clouds (assumes UTM projection).
    shadow_azimuth = ee.Number(90).subtract(ee.Number(img.get('MEAN_SOLAR_AZIMUTH_ANGLE')));

    # Project shadows from clouds for the distance specified by the CLD_PRJ_DIST input.
    cld_proj = (img.select('clouds').directionalDistanceTransform(shadow_azimuth, CLD_PRJ_DIST*10)
        .reproject(**{'crs': img.select(0).projection(), 'scale': 100})
        .select('distance')
        .mask()
        .rename('cloud_transform'))

    # Identify the intersection of dark pixels with cloud shadow projection.
    shadows = cld_proj.multiply(dark_pixels).rename('shadows')

    # Add dark pixels, cloud projection, and identified shadows as image bands.
    return img.addBands(ee.Image([dark_pixels, cld_proj, shadows]))

def add_cld_shdw_mask(img):
    # Add cloud component bands.
    img_cloud = add_cloud_bands(img)

    # Add cloud shadow component bands.
    img_cloud_shadow = add_shadow_bands(img_cloud)

    # Combine cloud and shadow mask, set cloud and shadow as value 1, else 0.
    is_cld_shdw = img_cloud_shadow.select('clouds').add(img_cloud_shadow.select('shadows')).gt(0)

    # Remove small cloud-shadow patches and dilate remaining pixels by BUFFER input.
    # 20 m scale is for speed, and assumes clouds don't require 10 m precision.
    is_cld_shdw = (is_cld_shdw.focalMin(2).focalMax(BUFFER*2/20)
        .reproject(**{'crs': img.select([0]).projection(), 'scale': 30})
        .rename('cloudmask'))

    # Add the final cloud-shadow mask to the image.
    return img_cloud_shadow.addBands(is_cld_shdw)

#s2_sr_cld_col_eval_disp = s2_sr_cld_col_eval.map(add_cld_shdw_mask)

# Mosaic the image collection
#img_mosaic = s2_sr_cld_col_eval_disp.mosaic()

def apply_cld_shdw_mask(img):
    # Subset the cloudmask band and invert it so clouds/shadow are 0, else 1.
    not_cld_shdw = img.select('cloudmask').Not()

    # Subset reflectance bands and update their masks, return the result.
    return img.select('B.*').updateMask(not_cld_shdw)


In [15]:
# DECISION RUNNING SENTINEL AND NDWI RELATED
if I_satellite == 'Sentinel':
    ndwi_hi = ndwi_hi_sentinel # for Sentinel

    s2_sr_cld_col = get_s2_sr_cld_col(AOI, START_DATE, END_DATE)

    SEN_BANDS = ['B2',   'B3', 'B4',  'B5', 'B6', 'B7', 'B8', 'B8A', 'B11', 'B12']
    bandNamesSentinel2 = ['blue', 'green', 'red', 'redE1', 'redE2', 'redE3', 'nir', 'redE4', 'swir1', 'swir2']

    s2_sr_median = (s2_sr_cld_col.map(add_cld_shdw_mask)
                                 .map(apply_cld_shdw_mask)
                                 .select(SEN_BANDS, bandNamesSentinel2)
                                 .median())
    # VISUALIZE SENTINEL CLOUDLESS
    Map.addLayer(s2_sr_median.clip(AOI),
                    {'bands': ['swir2', 'nir', 'red'], 'min': 0, 'max': 6000, 'gamma': 1.1},
                    'S2 cloud-free mosaic')

    Map.addLayer(treeLossYear.clip(AOI),{"opacity":1,"bands":["lossyear"],"min":1,"max":21,"palette":["3d358c","4457c9","4777f0","4196ff","2eb4f3","1ad1d5","1ae5b6","36f493","64fd6a","92ff47","b4f836","d3e935","ecd239","fbb938","fe992c","f9751d","ec520e","d93806","bf2102","9f1001","7a0403"]},"Tree Loss Year",False)
    Map.addLayer(minLoss.clip(AOI),{
        'palette': ['#ff0000']
    }, f'tree cover >30% loss since 20{year_start_loss}',False)
elif I_satellite == 'LANDSAT':
    ndwi_hi = ndwi_hi_landsat # for Landsat
    print('ndwi_landsat threshold',ndwi_hi )
    
else:
    ndwi_hi = ndwi_hi_planet
    print('ndwi_planet threshold',ndwi_hi )
     

ndwi_landsat threshold 0.1


In [16]:
# Function to use Sentinel data for generating Forest, from NDVI, BI, SI
def applying_sentinel_forest(s2_sr_median,ndvi_hi,bi_hi,si_hi,pixels): 
    ############# SPECTRAL INDICES #############################
    #NDVI - NORMALIZED DIFFERENCES VEGETATION INDEX
    ndvi = s2_sr_median.normalizedDifference(['nir','red'])
    #BARELAND/ BARE SOIL INDEX
    bi_1 = s2_sr_median.expression(
        '(NIR + GREEN + RED) / (NIR + GREEN - RED)', 
            {
          'NIR': s2_sr_median.select('nir').divide(10000),
          'GREEN': s2_sr_median.select('green').divide(10000),
          'RED': s2_sr_median.select('red').divide(10000)
            })
    #SOIL INDEX
    si = s2_sr_median.expression(
         '((1 - GREEN) * (1 - RED) )**(0.5)' ,
        {
          'GREEN': s2_sr_median.select('green').divide(10000),
          'RED': s2_sr_median.select('red').divide(10000)
        })

    #Map.addLayer(ndvi,{},'NDVI')
    #Map.addLayer(bi_1,{},'BI_1')
    #Map.addLayer(si,{},'SI')
    ############################################################

    ########### DECISION ON THE DEFINITION
    # NDVI > NDVI_High threshold
    ndviHi_masked = ndvi.updateMask(ndvi.gt(ndvi_hi))
    # BI < BI_High threshold
    bi_1Lo_masked = bi_1.updateMask(bi_1.lt(bi_hi))
    # Shadow Index (SI)> SI threshold
    siHi_masked = si.updateMask(si.gt(si_hi))

    #### HIGH DENSITY FOREST DECISION
    # If ndvi is high enough, baresoil index is low and shadow index is high, then it is a high density forest based on this
    hiforest_masked = ndviHi_masked.And(bi_1Lo_masked).And(siHi_masked)
    #Create connectedPixelCount() to get contiguous area, minimum mapping unit
    conthiForest = hiforest_masked.connectedPixelCount()
    #Apply the minimum area requirement, for the final result one area should have at least 
    hiforest_masked = conthiForest.gte(pixels).selfMask()
    
    #Map.addLayer(hiforest_masked.clip(AOI),{'palette':hi_forest},'High Forest',False)

    '''
    #Low density Forest - not used anywhere
    ndviMid_masked = ndvi.updateMask(ndvi.lt(ndvi_hi).And(ndvi.gt(ndvi_lo)))
    siMid_masked = si.updateMask(si.lt(si_hi).And(si.gt(si_lo)))

    loforest_masked = ndviMid_masked.And(bi_1Lo_masked).And(siMid_masked)
    #Map.addLayer(loforest_masked,{'palette':low_forest},'Low Forest')
    '''
    return hiforest_masked.clip(AOI)


In [17]:
# Utils
def assigning_band(band_name_image,class_value,srcImg):
        
    # Create an image with the constant value for class
    constant_image_class = srcImg.multiply(0).add(class_value).rename(band_name_image)
    constant_image_pixel = srcImg.multiply(0).add(1).rename('pixel')
    
    # Add the new band to the existing image
    pixel_bandimg= srcImg.addBands(constant_image_pixel)
    pix_classImg = pixel_bandimg.addBands(constant_image_class)
    pix_classImg = pix_classImg.select([band_name_image,'pixel'])
    
    return pix_classImg

In [18]:
## VERSION 1 for Sentinel as forest basemap, and followed by Hansen data - in function ####################################
def applying_sentinel_water(s2_sr_median, ndwi_hi):

    # Normalized Differences Water Index
    ndwi = s2_sr_median.normalizedDifference(['green','nir'])
    #Map.addLayer(ndwi,{},'NDWI')

    # Mask the non-watery parts of the image, where NDWI < ndwi_hi.
    waterMasked = ndwi.updateMask(ndwi.gt(ndwi_hi))
    #Map.addLayer(waterMasked, {'palette': water}, 'NDWI (Water) masked')
    
    ##### Get the area only that is not possible for planting (Water)
    waterinAOI = waterMasked.And(AOI_img)
    #Map.addLayer(waterinAOI,{'palette': water},"Water in AOI")
    water_edited = ee.Image(assigning_band(band_name_image,5,waterinAOI))
    return [waterinAOI,water_edited]

def applying_zone_sentinel_hansen(hiforest_masked,minLoss,s2_sr_median, ndwi_hi):
    
    ######### Zone 2 - High Baseline Forest: That does not have (overlaid with) TCL 
    # Unmasked the forest loss 10 years rule
    unmaskedLoss = AOI_img.unmask().updateMask(minLoss.mask().Not()).clip(AOI)
    highBaselineF = hiforest_masked.And(unmaskedLoss)
    #Map.addLayer(highBaselineF,{'palette':hi_forest},'High baseline (Forest)')
    
    # assigning class band 2
    highf_edited = ee.Image(assigning_band(band_name_image,2,highBaselineF)) #high baseline forest - 2
    ##################################################
    
    ######### Zone 3: High Forest and Loss (e.g., Young Regenerating Forest)
    ####### Get the overlay information of HighBaseline (Sentinel) and Tree Cover Loss (Hansen), e.g., Young Regenerating Forest
    HiForestAndLoss = AOI_img.And(hiforest_masked.And(minLoss)) #minLoss is the actual TCL without overlaying with Sentinel
    #Map.addLayer(HiForestAndLoss,{'palette': '#FF0000'},"High Baseline and not pass 10 years rule")
    
    # Final Result for High Forest and TCL -> Class number: 3
    tenyrfl_edited = ee.Image(assigning_band(band_name_image,3,HiForestAndLoss)) #tenyears rule not pass and high baseline - 3
    ##############################################
    
    ######### Zone 4: Tree Cover Loss on the current low baseline (Sentinel)
    ###### Get the 10 years data only that is not overlay with Sentinel High baseline (Forest) #############
    # Create a helper mask indicating where the smaller areas maskhiFL, distinguish only the highbaseline and TCL (mask) and assign mask as 1
    maskHiFL = HiForestAndLoss.mask()
    #Map.addLayer(maskHiFL)
    # Helper - Invert the mask to get the areas where the smaller raster is absent - get only area outside high forest and loss, and mask from 1 to 0
    # invert mask just to get the number from 0 to 1 or vice versa -> .Not()
    maskHiFL_inverted = maskHiFL.Not()
    #Map.addLayer(maskHiFL_inverted)
    # Unmask the bigger raster in the areas to get the pixel value for area 'outside' HiFL (High Baseline and Forest Loss)
    unmaskedHiFL = AOI_img.unmask().updateMask(maskHiFL_inverted).clip(AOI)
    #Map.addLayer(unmaskedHiFL)
    #outcome result for 10 years data only that is not overlay with Sentinel High baseline (Forest) unmaskedHiFL is Area that is not Forest Sentinel############
    tenYearsRule = unmaskedHiFL.And(minLoss)
    #Map.addLayer(tenYearsRule,{'palette': '#FFA500'},"10 Years Rule - not OK")
    
    # assign the Class into no. 4 - Final Result
    tenyrule_edited = ee.Image(assigning_band(band_name_image,4,tenYearsRule)) #tenyears rule not pass - 4
    ###############################################
    
    ######### Zone 5: Water Body - NDWI Sentinel
    # from a function, output is already assigned in a band name: Class : 5
    #water_list = applying_sentinel_water(s2_sr_median, ndwi_hi)
    #waterinAOI = water_list[0]
    #water_edited = water_list[1]
    
    # Use hansen data instead - better result, no need to adjust threshold and water body not change for long time
    WaterMask = gfc.select(['datamask']).rename('datamask').eq(2)
    waterinAOI = AOI_img.mask().updateMask(WaterMask)
    water_edited = ee.Image(assigning_band(band_name_image,5,waterinAOI)).clip(AOI)
    # unmasked the water (hansen)
    unmaskedWaterAOI = AOI_img.unmask().updateMask(waterinAOI.mask().Not()).clip(AOI)
    
    ###############################################


    ######### Zone 1: Go Zone - outside (forest, forest - TCL, no-forest TCL, WaterBody)
    # Utils - Similar with above (but easier), unmasked the high forest means that in the end only include area that is not in forest (hiforest_masked)
    maskHiF = hiforest_masked.mask()
    maskHiF_inverted = maskHiF.Not()
    # Unmasked High Forest - the result is all the area outside of hiforest_masked (Total all forest), and now in no forest
    unmaskedHiF = AOI_img.unmask().updateMask(maskHiF_inverted).clip(AOI)
    # unmasked the water
    #unmaskedWaterAOI = AOI_img.unmask().updateMask(waterinAOI.mask().Not()).clip(AOI)
    
    goZone = unmaskedLoss.And(unmaskedHiF).And(unmaskedWaterAOI)
    #Map.addLayer(goZone,{'palette':'#FFFF00'},'Go Zone')
    # assigning band - Go Zone
    goZone_edited = ee.Image(assigning_band(band_name_image,1,goZone)) #go Zone - 1
    ##############################################
    
    Map.addLayer(goZone_edited,{'bands': ['Class'],'palette':'#FFFF00'},'Go Zone')
    Map.addLayer(highf_edited,{'bands': ['Class'],'palette':hi_forest},'High baseline (Forest)')
    Map.addLayer(tenyrfl_edited,{'bands': ['Class'],'palette': '#FF0000'},"High Baseline and not pass 10 years rule")
    Map.addLayer(tenyrule_edited,{'bands': ['Class'],'palette': '#FFA500'},"10 Years Rule - not OK")
    Map.addLayer(water_edited,{'bands': ['Class'],'palette': water},"Water in AOI")
    
    labels = ['High Baseline (Forest)', '10 Years Rule not OK', 'High Baseline not passed 10 years rule', 'Water', 'Go Zone']
    # colorS can be defined using either hex code or RGB (0-255, 0-255, 0-255)
    colors = ['#006837', '#FFA500', '#FF0000', '#3380cc', '#FFFF00']
    # legend_colors = [(255, 0, 0), (127, 255, 0), (127, 18, 25), (36, 70, 180), (96, 68 123)]

    Map.add_legend(title='Legend', labels=labels, colors=colors)

    ''' mosaicking into single image  - still does not work
    #image_list = ee.List([goZone_edited, highf_edited, tenyrfl_edited, tenyrule_edited, water_edited ])
    # Extract the spatial information from one of the individual images.
    projection = goZone_edited.projection()
    scale = goZone_edited.projection().nominalScale()

    # Create an ee.ImageCollection from the list of images
    image_collection = ee.ImageCollection([goZone_edited, highf_edited, tenyrfl_edited, tenyrule_edited, water_edited ])

    # Merge the images into a single ee.Image
    zoning_final = image_collection.mosaic()
    '''
    return [goZone_edited,highf_edited,tenyrfl_edited,tenyrule_edited, water_edited]

def apply_arief_equation(s2_sr_median, threshold):
    # equation selected: Y = e**(43.2 - 380 X1 – 0.00970 X2 + 0.916 X3 + 6.5 X4- 30.6 X5 + 259 X6)
    # equation2 selected: Y = 6196 - 64212 X1 - 1.29 X2 + 120.0 X3 + 954 X4- 4547 X5 + 43814 X6
    # X1 = NDVI = (NIR - RED)/ (NIR + RED)
    # X2 = DVI = NIR – Red
    # X3 = RDVI = (NIR - Red)/((NIR + RED)**0.5)  
    # X4 = SR = NIR/Red 
    # X5 = MSRVI = ((NIR/Red) - 1)/((NIR/Red)**0.5 - 1)
    # X6 = SAVI = ((NIR - RED)/(NIR + RED + 0.5))*1.5

    #NDVI - NORMALIZED DIFFERENCES VEGETATION INDEX
    ndvi = s2_sr_median.normalizedDifference(['nir','red']).rename('ndvi')
    
    # DVI
    dvi = s2_sr_median.expression(
        '(NIR - RED)', 
            {
          'NIR': s2_sr_median.select('nir'),#.divide(10000),
          'RED': s2_sr_median.select('red'),#.divide(10000)
            }).rename('dvi')

    
    # RDVI
    rdvi = s2_sr_median.expression(
        '(NIR - Red)/((NIR + Red)**0.5)',
            {
            'NIR': s2_sr_median.select('nir'),#.divide(10000),
            'Red': s2_sr_median.select('red'),#.divide(10000)
            }
        ).rename('rdvi')
    
    # SR - Simple Ratio
    sr_index = s2_sr_median.expression(
        'NIR/Red',
            {
            'NIR': s2_sr_median.select('nir'),#.divide(10000),
            'Red': s2_sr_median.select('red'),#.divide(10000)
            }
        ).rename('sr_index')
    
    # MSRVI
    msrvi = s2_sr_median.expression(
        '((NIR/Red) - 1)/((NIR/Red)**0.5 - 1)',
            {
            'NIR': s2_sr_median.select('nir'),#.divide(10000),
            'Red': s2_sr_median.select('red'),#.divide(10000)
            }
        ).rename('msrvi')
    
    # SAVI
    savi = s2_sr_median.expression(
        '((NIR - RED)/(NIR + RED + 0.5))*1.5',
            {
            'NIR': s2_sr_median.select('nir'),#.divide(10000),
            'RED': s2_sr_median.select('red'),#.divide(10000)
            }
        ).rename('savi')
    
    all_spec_index = ndvi.addBands(ee.Image(dvi)) \
                                 .addBands(ee.Image(sr_index)) \
                                 .addBands(ee.Image(msrvi)) \
                                 .addBands(ee.Image(rdvi)) \
                                 .addBands(ee.Image(savi))
                        
    e_num = math.e
    # print(e_num)
    # from arief et al, https://journal.itera.ac.id/index.php/jsat/article/view/964/348
    arief_etal = all_spec_index.expression(
            'e**(43.2 - (380* X1) - (0.00970* X2) + (0.916* X3) + (6.5* X4) - (30.6* X5) + (259* X6))',
            #'6196 - (64212 * X1) - (1.29 * X2) + (120.0 * X3) + (954 * X4)- (4547 * X5) + (43814 * X6)',
            {
                'e': e_num,
                'X1': all_spec_index.select('ndvi'),
                'X2': all_spec_index.select('dvi'),
                'X3': all_spec_index.select('rdvi'),
                'X4': all_spec_index.select('sr_index'),
                'X5': all_spec_index.select('msrvi'),
                'X6': all_spec_index.select('savi'),
            }
        ).rename('y_regression')
    #Map.addLayer(all_spec_index.clip(AOI),{},'INDEXXXX_ALL')
    #Map.addLayer(arief_etal.clip(AOI),{},'y')
    spec_forest = arief_etal.gte(threshold).selfMask()
    #Map.addLayer(spec_forest.clip(AOI),{},'y_forest')
    return spec_forest
    

In [19]:
def eval_accuracy(band_names_ref, band_name_image,high_forest_image ):
    
    ref_values = gdf_ref[band_names_ref].values
    print("Reference_values: ", ref_values)
    # Select the band of interest from the image to know what classified 1 Forest (High baseline), 0 (outside)
    #band_name_image = 'Class'
    band = high_forest_image.select([band_name_image])
    
    # Unmask the band, replacing masked values (null) - Class: null with 0
    unmasked_band = band.unmask(0)
    
    # Use sampleRegions to extract pixel values at the point locations
    sampled_points = unmasked_band.sampleRegions(
        collection=ref_point_loc,
        scale=10
    )

    # Get the pixel values as a list - another way to get the ref_values
    # ref_values = sampled_points.aggregate_array('Forest_Sen').getInfo()

    # Get the pixel values as a list
    pixel_values = sampled_points.aggregate_array(band_name_image).getInfo()

    # Print the pixel values
    print("Pixel Values: ", pixel_values)
    
    confusion_mat = confusion_matrix(ref_values, pixel_values)
    print("Confusion matrix summary: \n", confusion_mat)
    
    tn, fp, fn, tp = confusion_matrix(ref_values, pixel_values).ravel()
    
    print("TN:",tn,"\n FP:",fp, "\n FN:",fn, "\n TP:",tp)
    
    accuracy = ((tp + tn) / (tp + tn + fp + fn))*100.00
    print("Accuracy: (%) ",accuracy)
    
    precision = (tp / (tp + fp))*100.00
    print("Precision: (%) ",precision)
    
    prod_acc = (tp / (tp + fn))*100.00
    print("Producer Accuracy: (%) ",prod_acc) #means that how is the raster result (image) predict forest only,
    
    user_acc = (tn / (tn + fp))*100.00
    print("User Accuracy: (%) ",user_acc)
    return [tn, fp, fn, tp]
    

In [20]:
''' What confusion matrix look like
                 Predicted Class
               | Forest (1) | No Forest (0) |
True Class     |------------|--------------|
Forest (1)     |    TP      |      FN      |
No Forest (0)  |    FP      |      TN      |

Here is the definition of accuracy and precision

Accuracy: The overall accuracy of the classification.

Accuracy = (TP + TN) / (TP + TN + FP + FN)

Precision: The proportion of correctly classified positive samples (class 1) out of all samples classified as positive.

Precision = TP / (TP + FP)

Producer Accuracy (also known as Recall or Sensitivity): The proportion of correctly classified positive samples (class 1) out of all actual positive samples.

Producer Accuracy = TP / (TP + FN)

User Accuracy (also known as Specificity): The proportion of correctly classified negative samples (class 0) out of all actual negative samples.

User Accuracy = TN / (TN + FP)

'''

' What confusion matrix look like\n                 Predicted Class\n               | Forest (1) | No Forest (0) |\nTrue Class     |------------|--------------|\nForest (1)     |    TP      |      FN      |\nNo Forest (0)  |    FP      |      TN      |\n\nHere is the definition of accuracy and precision\n\nAccuracy: The overall accuracy of the classification.\n\nAccuracy = (TP + TN) / (TP + TN + FP + FN)\n\nPrecision: The proportion of correctly classified positive samples (class 1) out of all samples classified as positive.\n\nPrecision = TP / (TP + FP)\n\nProducer Accuracy (also known as Recall or Sensitivity): The proportion of correctly classified positive samples (class 1) out of all actual positive samples.\n\nProducer Accuracy = TP / (TP + FN)\n\nUser Accuracy (also known as Specificity): The proportion of correctly classified negative samples (class 0) out of all actual negative samples.\n\nUser Accuracy = TN / (TN + FP)\n\n'

In [21]:
# ADDING VISUAL FOR CHECKING AND VERIFY THE RESULT
visPlanet_veg = {"bands":["red","nir","blue"],"min":0,"max":0.6,"gamma":1.5}
truePlanet_veg = {"bands":["red","green","blue"],"min":0,"max":0.6,"gamma":1.5}

Map.addLayer(filtered_scaled, visPlanet_veg , 'NICFI Cloudless Image - RNB')
Map.addLayer(filtered_scaled, truePlanet_veg , 'NICFI Cloudless Image - RGB')

In [22]:
if algor_process == 'fcd_galcit':
    if I_satellite == 'LANDSAT':
        print('selecting LANDSAT images')
        date = [START_DATE, END_DATE]
        date_object = datetime.strptime(END_DATE, "%Y-%m-%d")
        year = date_object.year

        if year < 2014:
            landsat = landsat457
            landsat_raw = landsat_raw457
        else:
            landsat = landsat89
            landsat_raw = landsat_raw89
        image_col = landsat(AOI, date)
        image_uncorrected = landsat(AOI, date).median().clip(AOI)
        image_corrected = image_col.map(doPhysicalCorrection).median().clip(AOI)
        Map.addLayer(image_uncorrected,{'bands': ['swir2', 'nir', 'red'], 'min': 0, 'max': 0.6, 'gamma': 1.5 }, f'landsat_for_FCD_{year}_uncorrected')
        Map.addLayer(image_corrected,{'bands': ['swir2', 'nir', 'red'], 'min': 0, 'max': 0.6, 'gamma': 1.5 }, f'landsat_for_FCD_{year}_corrected')
    
    elif I_satellite == 'PlanetNicfi':
        print('selecting Planet NICFI images')
        image_corrected = filtered_scaled
        
    print('processing AVI')
    avi_image = AVI_func(image_corrected)
    #Map.addLayer(avi_image,{},'avi')
    
    print('processing BSI')
    bsi_image = BSI_func(image_corrected)
    #Map.addLayer(bsi_image,{},'bsi')
    
    print('processing SI')
    si_image = SI_func(image_corrected)
    #Map.addLayer(si_image,{},'si_no_norm')
        
    #Map.addLayer(image_corr_8,{'bands': ['swir2', 'nir', 'red'], 'min': 0, 'max': 256,  }, f'landsat_for_FCD_{year}_corrected_8bit')        
    print('Normalizing to 100 AVI')
    # Starting (normalized 100) Indices
    avi_norm = normalization_100(avi_image)
    Map.addLayer(avi_norm,{'min':0,'max':100},'avi_norm')
    print('Normalizing to 100 BSI')
    bsi_norm = normalization_100(bsi_image)
    Map.addLayer(bsi_norm,{'min':0,'max':100},'bsi_norm')
    print('Normalizing to 100 SI')
    si_norm = normalization_100(si_image)
    Map.addLayer(si_norm, {'min':0,'max':100},'si_norm')
    print('Combining AVI AND BSI')
    # Combine  AVI and BSI to one image with two bands 
    AVI_BSI = avi_norm.addBands(bsi_norm)
    # Masked-out process or remove null data, to avoid errors
    avi_bsi_clean = AVI_BSI.gte(0).Or(AVI_BSI.lte(0))
    AVI_BSI = AVI_BSI.updateMask(avi_bsi_clean)
    #Map.addLayer(AVI_BSI, {'min':0,'max':100}, 'AVI_BSI')
    
    print('Processing means center of AVI_BSI please wait')
    # Means Centered for VD --- VD WILL GIVE YOU ERROR ---> if you dont masked out first
    Means_cn = means_centered(AVI_BSI, AOI)
    #print('Means_cn[0]',Means_cn[0])
    #print('Means_cn[1]',Means_cn[1])
    
    print('Now we get to the PCA of Vegetation density')
    # PCA FOR AVI AND BSI = VD and SVI
    VD = getPrincipalComponents(Means_cn[0], pca_scale, AOI, Means_cn[1]).rename(['VD1', 'VD2'])
    #Map.addLayer(VD,{},'VD')
    # scale VD datasets 
    SVI = normalization_100(VD)
    Map.addLayer(SVI,{'min':0,'max':100},'SVI')
    print('Success get the PCA normalized of VD => SVI')
    
    
    if TI_include: 
        #image_raw = landsat_raw(AOI, date).median().clip(AOI)
        image_col_raw = landsat_raw(AOI, date)

        # Define the weighted median function
        def weighted_median(collection):
            # Sort the collection by the timestamp in descending order
            sorted_collection = collection.sort('system:time_start', False)

            # Compute the weighted median
            weighted_median = sorted_collection.median()

            return weighted_median

        # Compute the weighted median
        image_raw = weighted_median(image_col_raw).clip(AOI)
        #image_raw = image_col_raw.median().clip(AOI)

        #Map.addLayer(image_raw, {'bands': ['swir2', 'nir', 'red'], 'min': 0, 'max': 20000, 'gamma': 1.5 },'image_raw')

        ti_norm = normalization_100(image_raw.select('TI'))
        Map.addLayer(ti_norm,{'min':0,'max':100},'TI_norm')
        #Map.addLayer(image_raw,{'bands': ['swir2', 'nir', 'red'], 'min': 0, 'max': 6, 'gamma': 1.5 },'ti_image')

        # Combine SI and TI to one image with two bands
        SI_TI = si_norm.addBands(ti_norm)
        #Map.addLayer(SI_TI, {'min':0,'max':100}, 'SI_TI')

        print('Processing means center of SI and TI please wait')
        # Means centered FOR SI AND TI (NORMALIZED)
        # Means Centered for SSI --- SSI WILL GIVE YOU ERROR ---> if you dont masked out first
        Means_cn_siti = means_centered(SI_TI, AOI) 
        #print('Means_cn_siti[0]',Means_cn_siti[0])
        #print('Means_cn_siti[1]',Means_cn_siti[1])

        print('Now we get to the PCA of SI and TI')
        # PCA FOR AVI AND BSI = VD and SVI
        SI_TI_PCA = getPrincipalComponents(Means_cn_siti[0], pca_scale, AOI, Means_cn_siti[1]).rename(['SSI1', 'SSI2'])
        SSI = normalization_100(SI_TI_PCA)
        #Map.addLayer(SI_TI_PCA,{}, 'SI_TI_PCA')
        Map.addLayer(SSI,{'min':0,'max':100},'SSI')
        print('Success get the PCA normalized of SI and TI => SSI')
        
        print('Now calculating the FCD from SVI and SSI - selecting band svi1 svi2 ssi1 and ssi2')
        svi1 = SVI.select(['VD1'])
        ssi1 = SSI.select(['SSI1'])

        # Starting to include in the formula for Forest Cover Density
        FCD1_1 = (((svi1.multiply(ssi1)).add(1)).pow(0.5)).subtract(1)
        Map.addLayer(FCD1_1, {'min':0 ,'max':80, 'palette':['ff4c16', 'ffd96c', '39a71d']}, 'FCD1_1')
        #Map.addLayer(FCD1.mask(FCD1.gte(30)), {'min':0 ,'max':80, 'palette':['ff4c16', 'ffd96c', '39a71d']}, 'FCD1_morethan30')

        svi2 = SVI.select(['VD2'])
        ssi2 = SSI.select(['SSI2'])

        FCD2_2 = (((svi2.multiply(ssi2)).add(1)).pow(0.5)).subtract(1)

        Map.addLayer(FCD2_2, {'min':0 ,'max':80, 'palette':['ff4c16', 'ffd96c', '39a71d']}, 'FCD2_2')
        #Map.addLayer(FCD2_2.mask(FCD2_2.gte(30)), {'min':0 ,'max':80, 'palette':['ff4c16', 'ffd96c', '39a71d']}, 'FCD2_2_morethan30')

        FCD1_2 = (((svi1.multiply(ssi2)).add(1)).pow(0.5)).subtract(1)

        Map.addLayer(FCD1_2, {'min':0 ,'max':80, 'palette':['ff4c16', 'ffd96c', '39a71d']}, 'FCD1_2')
        #Map.addLayer(FCD1_2.mask(FCD1_2.gte(30)), {'min':0 ,'max':80, 'palette':['ff4c16', 'ffd96c', '39a71d']}, 'FCD1_2_morethan30')

        print('Now calculating the FCD from SVI2 and SSI1 into FCD in percentage')
        # The best on AOI_B30 based on the visual check, combination svi2 and ssi1
        FCD2_1 = (((svi2.multiply(ssi1)).add(1)).pow(0.5)).subtract(1)

        Map.addLayer(FCD2_1, {'min':0 ,'max':80, 'palette':['ff4c16', 'ffd96c', '39a71d']}, 'FCD2_1')
        #Map.addLayer(FCD2_1.mask(FCD2_1.gte(30)), {'min':0 ,'max':80, 'palette':['ff4c16', 'ffd96c', '39a71d']}, 'FCD2_1_morethan30')
        print('finish processing PCA please continue')
        
    else:
        print('no thermal band, choosing planet lab nicfi images')
        si = si_norm
        #minsi =si.reduceRegion(reducer=ee.Reducer.min(),geometry=AOI,scale=1000,bestEffort=True,tileScale=16)
        #minsi = ee.Dictionary(minsi).toImage()
        #maxsi =si.reduceRegion(reducer=ee.Reducer.max(),geometry=AOI,scale=1000,bestEffort=True,tileScale=16)
        #maxsi = ee.Dictionary(maxsi).toImage()

        #ssiscale = si.subtract(minsi).divide(maxsi.subtract(minsi)).multiply(100)
        #print(ssiscale, 'SSI')
        Map.addLayer(si,{'min':0, 'max':100}, 'SSI')

        print('Now calculating the FCD from SVI and SSI - selecting band svi1 svi2 ssi1 and ssi2')
        svi1 = SVI.select(['VD1'])
        ssi1 = si.select(['SI'])
        svi2 = SVI.select(['VD2'])

        # Starting to include in the formula for Forest Cover Density
        FCD1_1 = (((svi1.multiply(ssi1)).add(1)).pow(0.5)).subtract(1)
        Map.addLayer(FCD1_1, {'min':0 ,'max':80, 'palette':['ff4c16', 'ffd96c', '39a71d']}, 'FCD1_1')
        #Map.addLayer(FCD1.mask(FCD1.gte(30)), {'min':0 ,'max':80, 'palette':['ff4c16', 'ffd96c', '39a71d']}, 'FCD1_morethan30')


        print('Now calculating the FCD from SVI2 and SSI1 into FCD in percentage')
        # The best on AOI_B30 based on the visual check, combination svi2 and ssi1
        FCD2_1 = (((svi2.multiply(ssi1)).add(1)).pow(0.5)).subtract(1)

        Map.addLayer(FCD2_1, {'min':0 ,'max':80, 'palette':['ff4c16', 'ffd96c', '39a71d']}, 'FCD2_1')
        #Map.addLayer(FCD2_1.mask(FCD2_1.gte(30)), {'min':0 ,'max':80, 'palette':['ff4c16', 'ffd96c', '39a71d']}, 'FCD2_1_morethan30')
        print('finish processing PCA please continue')
    

selecting LANDSAT images
processing AVI
processing BSI
processing SI
Normalizing to 100 AVI
Normalizing to 100 BSI
Normalizing to 100 SI
Combining AVI AND BSI
Processing means center of AVI_BSI please wait
Now we get to the PCA of Vegetation density
Success get the PCA normalized of VD => SVI
Processing means center of SI and TI please wait
Now we get to the PCA of SI and TI
Success get the PCA normalized of SI and TI => SSI
Now calculating the FCD from SVI and SSI - selecting band svi1 svi2 ssi1 and ssi2
Now calculating the FCD from SVI2 and SSI1 into FCD in percentage
finish processing PCA please continue


In [23]:
## BASELINE SELECTION FOREST
# run if arief et al for main data, or additional of hansen based
if algor_process == 'Sentinel' or algor_process == 'Hansen':
    hiforest_masked = applying_sentinel_forest(s2_sr_median,ndvi_hi,bi_hi,si_hi,pixels)

# run if arief et al for main data, or additional of hansen based
if algor_process == 'arief_hansen' or algor_process == 'hansen_arief':
    image_y_highbaseline = apply_arief_equation(s2_sr_median, y_threshold)

# processing if fcd used    
if algor_process == 'fcd_galcit':
    FCD = FCD2_1
    
    print('Success calculating FCD best combination, and added to the map')
    print('Now we label the FCD')
    # Starting to create threshold for labeling
    WaterMask = gfc.select(['datamask']).rename('datamask').eq(2)
    WaterinAOI = AOI_img.mask().updateMask(WaterMask)
    # unmasked the water (hansen)
    unmaskedWaterAOI = AOI_img.unmask().updateMask(WaterinAOI.mask().Not()).clip(AOI)
    
    ######### Zone 5: Water Body - NDWI Sentinel
    # from a function, output is already assigned in a band name: Class : 5
    #water_list = applying_sentinel_water(image_corrected, ndwi_hi)
    #waterinAOI_ndwi = water_list[0]
    #unmaskedWaterAOI_ndwi = AOI_img.unmask().updateMask(waterinAOI_ndwi.mask().Not()).clip(AOI)
    
    
    print(f'Adding the map of Forest, FCD >= {yrf_forest}% and mask only if not water in area (hansen) and NDWI')
    #Forest
    AllForest = FCD.mask(FCD.gte(yrf_forest).And(unmaskedWaterAOI))
    HighForestDense = FCD.mask(FCD.gte(high_forest).And(unmaskedWaterAOI))
    YRFForestDense = FCD.mask(FCD.gte(yrf_forest).And(unmaskedWaterAOI).And(FCD.lt(high_forest)))
    Map.addLayer(HighForestDense, {'min':0 ,'max':100}, f'Forest (FCD >= {high_forest}%)')
    
    Map.addLayer(YRFForestDense, {'min':0 ,'max':100}, f'Forest (FCD >= {yrf_forest}%) and < {high_forest}%')
    
    print(f'Adding the map of Shrubland, FCD <  {yrf_forest}% and FCD >= {shrub_grass}%')
    #Shrubland
    Shrubland = FCD.mask(FCD.lt(yrf_forest).And(FCD.gte(shrub_grass)).And(unmaskedWaterAOI))
    Map.addLayer(Shrubland, {'min':0 ,'max':100}, f'Shrubland ({shrub_grass}% <= FCD < {yrf_forest}%)')
    
    print(f'Adding the map of Grassland or Openland, FCD  < {shrub_grass}%')
    #Grassland / openland
    Grassland = FCD.mask(FCD.lt(shrub_grass).And(unmaskedWaterAOI))
    Map.addLayer(Grassland, {'min':0 ,'max':100}, f'Grassland (FCD < {shrub_grass})')
    
    hiforest_masked = AllForest
    #Shrubland   
    #Grassland
    
    print('Processing - the zoning classification')
    highForestOnly = ee.Image(assigning_band(band_name_image,1,hiforest_masked.clip(AOI)))
    Map.addLayer(highForestOnly,{'bands': ['Class'],'palette':hi_forest},f'ALL_Forest_FCD > {yrf_forest}% - Class 1')
    tiff_result = applying_zone_sentinel_hansen(hiforest_masked,minLoss,image_corrected, ndwi_hi)
    band_names_ref = 'Forest_Lan' #this band_names refer to the name of column in shapefile - which column that refer as point reference value
    if eval_process: 
        eval = eval_accuracy(band_names_ref, band_name_image,highForestOnly)
    
    print('Process finish kindly check the map')
    


Success calculating FCD best combination, and added to the map
Now we label the FCD
Adding the map of Forest, FCD >= 50% and mask only if not water in area (hansen) and NDWI
Adding the map of Shrubland, FCD <  50% and FCD >= 30%
Adding the map of Grassland or Openland, FCD  < 30%
Processing - the zoning classification
Process finish kindly check the map


In [24]:
#########
    
# Starting process every other Option algorithm beside FCD
if algor_process == 'Sentinel':
    highForestOnly = ee.Image(assigning_band(band_name_image,1,hiforest_masked.clip(AOI)))
    Map.addLayer(highForestOnly,{'bands': ['Class'],'palette':hi_forest},'High_Forest_SENTINEL_Assigned - Class 1')
    tiff_result = applying_zone_sentinel_hansen(hiforest_masked,minLoss,s2_sr_median, ndwi_hi)
    band_names_ref = 'Forest_Sen' #this band_names refer to the name of column in shapefile - which column that refer as point reference value
    if eval_process: 
        eval = eval_accuracy(band_names_ref, band_name_image,highForestOnly)
    
elif algor_process == 'Hansen':
    band_names_ref = 'Forest_Lan' #this band_names refer to the name of column in shapefile - which column that refer as point reference value
     
    ## Version 2 - Hansen - Sentinel
    
    ## Zone 4: 10 Years rule not OK - > CC overlay with TCL, (Tree Crown Cover) more than the threshold in AOI
    tenYearsRule = minLoss.clip(AOI)
    tenYearsRule_edited = ee.Image(assigning_band(band_name_image,4,tenYearsRule))
    Map.addLayer(tenYearsRule_edited,{'bands': ['Class'],'palette': '#FFA500'},"10 Years Rule - not OK")
    
    
    ## Zone 3: Forest (Sentinel) passed 10 years rule not OK - TCL before year_start_loss overlay with > CC
    # Direct process to get the < year_rule TCL (Explicit)
    #treeLossYear = gfc.select(['lossyear'])
    treeLoss_lessYear = treeLossYear.lt(year_start_loss).selfMask()
    CC_lessYearloss_overlay = AOI_img.And(minArea.And(treeLoss_lessYear))
    # obtain area ten years < CC with the forest masked (Sentinel)
    ForestRegenerating_lessYear = hiforest_masked.And(CC_lessYearloss_overlay)
    ForestRegenerating_lessYear_edited = ee.Image(assigning_band(band_name_image,3,ForestRegenerating_lessYear))

    Map.addLayer(ForestRegenerating_lessYear_edited,{'bands': ['Class'],'palette': '#FF0000'},f"Forest (Sentinel) Regenerating - TCL < 20{year_start_loss}")
   
    ## Zone 5: Water Body Based on Hansen Data
    WaterMask = gfc.select(['datamask']).rename('datamask').eq(2)
    WaterAOI = AOI_img.mask().updateMask(WaterMask)
    # assigning band Class in WaterBody
    water_edited = ee.Image(assigning_band(band_name_image,5,WaterAOI))

    Map.addLayer(water_edited,{'bands': ['Class'],'palette': water},"Water in AOI")
    
    # unmasked the water
    unmaskedWaterAOI = AOI_img.unmask().updateMask(WaterAOI.mask().Not()).clip(AOI)
    
    # Zone 1: Go Zone - No Forest (Sentinel) < start_loss_year in CC
    hiFMasked = hiforest_masked.mask()
    maskedHiFI_inverted = hiFMasked.Not()
    unmaskedHiF = AOI_img.unmask().updateMask(maskedHiFI_inverted).clip(AOI)
    GoZoneNoForest_TCL_lessYear = unmaskedHiF.And(CC_lessYearloss_overlay).And(unmaskedWaterAOI)
    GoZoneNoForest_TCL_lessYear_edited = ee.Image(assigning_band(band_name_image,1,GoZoneNoForest_TCL_lessYear))

    Map.addLayer(GoZoneNoForest_TCL_lessYear_edited,{'bands': ['Class'],'palette': '#FFFF00'},"Go Zone - No Forest (Sentinel) in TCL less year >CC")

    ''' #Implying the same as year < year_start_loss and minArea
    ## Zone 3: Forest (Sentinel) 10 years rule not OK - TCL Overlaid with < CC, (Tree Crown Cover) less than the threshold in AOI
    # helper - mask
    tenYearsRule_mask = tenYearsRule.mask()
    masktenYearsRule_inverted = tenYearsRule_mask.Not()
    # Unmasked ten years all - all area outside ten years rule (> CC)
    unmaskedtenYrRuleALL = AOI_img.unmask().updateMask(masktenYearsRule_inverted).clip(AOI)
    # tree loss in the AOI
    tenYearsRuleAll = AOI_img.And(treeLoss)
    # obtain the area in outside ten years rule (>CC) intersected with all ten years rule -> tcl in < CC
    tenYearsRule_notCC = tenYearsRuleAll.And(unmaskedtenYrRuleALL)
    # obtain area ten years < CC with the forest masked (Sentinel)
    Forest10Years_noCC = hiforest_masked.And(tenYearsRule_notCC)
    Forest10Years_noCC_edited = ee.Image(assigning_band(band_name_image,3,Forest10Years_noCC))

    Map.addLayer(Forest10Years_noCC_edited,{'bands': ['Class'],'palette': '#32a844'},"Forest (Sentinel) 10 Years < CC")


    # Zone 1: Go Zone - No Forest (Sentinel) in TCL < CC
    hiFMasked = hiforest_masked.mask()
    maskedHiFI_inverted = hiFMasked.Not()
    unmaskedHiF = AOI_img.unmask().updateMask(maskedHiFI_inverted).clip(AOI)
    GoZoneNoForest_TCLnoCC = unmaskedHiF.And(tenYearsRule_notCC)
    GoZoneNoForest_TCLnoCC_edited = ee.Image(assigning_band(band_name_image,1,GoZoneNoForest_TCLnoCC))

    Map.addLayer(GoZoneNoForest_TCLnoCC_edited,{'bands': ['Class'],'palette': '#FFFF00'},"Go Zone - No Forest (Sentinel) in TCL < CC")
    '''
    
    ## Zone 2: Forest (Hansen > CC year 2000 Baseline) does not overlay with TCL (ALL)
    # helper - unmasked, get area only outside all the ten years rule TCL (ALL)
    # tree loss in the AOI and exclude no loss
    treeLossAll = treeLossYear.gt(0).selfMask()
    #Map.addLayer(treeLossAll,{},'treelossall', False)
    tenYearsRuleAll = AOI_img.And(treeLossAll)
    masked_tenYearsRuleAll = tenYearsRuleAll.mask()
    masked_inverted_tenYearsRuleAll = masked_tenYearsRuleAll.Not()
    unmasked_tenYearsRuleAll = AOI_img.unmask().updateMask(masked_inverted_tenYearsRuleAll).clip(AOI)
    # get the area that is forest in 2000 and has no TCL (No overlay with TCL ALL)
    Forest_RecentHansen = ForestArea2000Hansen.And(unmasked_tenYearsRuleAll)
    # assigning the number Class - 2
    Forest_RecentHansen_edited = ee.Image(assigning_band(band_name_image,2,Forest_RecentHansen))
    Map.addLayer(Forest_RecentHansen_edited,{'bands': ['Class'],'palette': hi_forest},f"Forest_{END_DATE} - Hansen (No TCL in Forest 2000 until {END_DATE}) to be evaluated")

    # Zone 3 - Forest (Sentinel) < CC (ALL TCL)
    # less than threshold
    canopyCover_lessThreshold = canopyCover.lt(cc).selfMask()
    Forest_Regenerating_lessCC = hiforest_masked.And(canopyCover_lessThreshold).And(AOI_img)
    Forest_Regenerating_lessCC_edited = ee.Image(assigning_band(band_name_image,3,Forest_Regenerating_lessCC))
    Map.addLayer(Forest_Regenerating_lessCC_edited,{'bands': ['Class'],'palette': '#FF0000'},f"Forest (Sentinel) CC < {tree_cover_forest}% - Regenerating")

    # Zone 1 - Go Zone area (No Forest (Sentinel) and < CC)
    GoZone_NoForestSen_lessThreshold = unmaskedHiF.And(canopyCover_lessThreshold).And(unmaskedWaterAOI)
    GoZone_NoForestSen_lessThreshold_edited = ee.Image(assigning_band(band_name_image,1,GoZone_NoForestSen_lessThreshold))
    Map.addLayer(GoZone_NoForestSen_lessThreshold_edited,{'bands': ['Class'],'palette': '#FFFF00'},"Go Zone - No Forest (Sentinel) < CC")
    

    if eval_process:
        print(f"EVALUATING ONLY IN FOREST AREA CC > {tree_cover_forest}% THAT GENERATED BY HANSEN 2000 THAT HAS NO TCL")
        ### EVALUATING FOREST CLASS HANSEN
        # CHANGE THE NUMBER Class into 1 as Forest for evaluation purpose only
        Forest_RecentHansen_edited_Class1_Forest = ee.Image(assigning_band(band_name_image,1,Forest_RecentHansen))
        print("Evaluating only Hansen Forest")
        eval = eval_accuracy(band_names_ref, band_name_image,Forest_RecentHansen_edited_Class1_Forest)   
        print("---------------------------- \n")

        ### Evaluating the all end Result Forest Classified (Hansen + Sentinel)

        def get_matrix(image, band_name_image):
            band = image.select([band_name_image])

            # Unmask the band, replacing masked values (null) - Class: null with 0
            unmasked_band = band.unmask(0)

            # Use sampleRegions to extract pixel values at the point locations
            sampled_points = unmasked_band.sampleRegions(
                collection=ref_point_loc,
                scale=10
            )

            pixel_values = sampled_points.aggregate_array(band_name_image).getInfo()

            return pixel_values

        ref_values = gdf_ref[band_names_ref].values

        # prepare for all class into 1, for matrix number assign as forest is 1, no forest is 0, assumed they are not overlapped
        print(f"Evaluating All Forest Classified (Hansen CC > {tree_cover_forest}%) + Sentinel (inGAP) vs Reference value")
        # in the end, if one of them classified forest, then the all result will be class: 1
        ForestRegenerating_lessYear_eval = ee.Image(assigning_band(band_name_image,1,ForestRegenerating_lessYear))   
        Forest_Regenerating_lessCC_eval = ee.Image(assigning_band(band_name_image,1,Forest_Regenerating_lessCC))

        #all_Forest_Hansen_Sentinel = [Forest_RecentHansen_edited_Class1_Forest,ForestRegenerating_lessYear_eval,Forest_Regenerating_lessCC_eval]

        HansenForestOnly_pixValue = get_matrix(Forest_RecentHansen_edited_Class1_Forest, band_name_image)
        print(f"Hansen pix no Loss in CC > {tree_cover_forest}%", HansenForestOnly_pixValue)
        ForestRegenerating_lessYear_pixValue = get_matrix(ForestRegenerating_lessYear_eval, band_name_image)
        print(f"Forest (Sentinel) pix in TCL year < 20{year_start_loss}",ForestRegenerating_lessYear_pixValue)
        Forest_Regenerating_lessCC_pixValue = get_matrix(Forest_Regenerating_lessCC_eval, band_name_image)
        print(f"Forest (Sentinel) pix in CC (year 2000) < {tree_cover_forest}% - Regenerating",Forest_Regenerating_lessCC_pixValue)
        print("------------------------------")

        a = np.array(HansenForestOnly_pixValue)
        b = np.array(ForestRegenerating_lessYear_pixValue)
        c = np.array(Forest_Regenerating_lessCC_pixValue)

        AllForestArr = a + b + c
        AllForestArr_list = AllForestArr.tolist()

        print("Reference Value: ", ref_values)
        print("Pixel Value (All Forest): ",AllForestArr_list)

        confusion_mat_all = confusion_matrix(ref_values, AllForestArr_list)
        print("Confusion matrix summary: \n", confusion_mat_all)

        tn, fp, fn, tp = confusion_matrix(ref_values, AllForestArr_list).ravel()

        print("TN:",tn,"\n FP:",fp, "\n FN:",fn, "\n TP:",tp)

        accuracy = ((tp + tn) / (tp + tn + fp + fn))*100.00
        print("Accuracy: (%) ",accuracy)

        precision = (tp / (tp + fp))*100.00
        print("Precision: (%) ",precision)

        prod_acc = (tp / (tp + fn))*100.00
        print("Producer Accuracy: (%) ",prod_acc) #means that how is the raster result (image) predict forest only,

        user_acc = (tn / (tn + fp))*100.00
        print("User Accuracy: (%) ",user_acc)
        
        
    labels = [f'Forest_{END_DATE} - Hansen (No TCL in Forest 2000 until {END_DATE})', 
          f'Forest (Sentinel) Regenerating - TCL < 20{year_start_loss}', 
          f"Forest (Sentinel) CC < {tree_cover_forest}% - Regenerating",
          "10 Years Rule - not OK",
          'Water', 
          'Go Zone',
         ]
    # colorS can be defined using either hex code or RGB (0-255, 0-255, 0-255)
    colors = [hi_forest, 
              '#FF0000', 
              '#FF0000', 
              '#FFA500', 
               water,
              '#FFFF00',
             ]
    # legend_colors = [(255, 0, 0), (127, 255, 0), (127, 18, 25), (36, 70, 180), (96, 68 123)]

    Map.add_legend(title='Legend', labels=labels, colors=colors)

elif algor_process == 'arief_hansen':
    highForestOnly = ee.Image(assigning_band(band_name_image,1,image_y_highbaseline.clip(AOI)))
    Map.addLayer(highForestOnly,{'bands': ['Class'],'palette':hi_forest},'High_Forest_SENTINEL_Assigned_Arief - Class 1')
    tiff_result = applying_zone_sentinel_hansen(image_y_highbaseline,minLoss,s2_sr_median, ndwi_hi)
    band_names_ref = 'Forest_Sen' #this band_names refer to the name of column in shapefile - which column that refer as point reference value
    if eval_process: 
        eval = eval_accuracy(band_names_ref, band_name_image,highForestOnly)
        
elif algor_process == 'hansen_arief':
    band_names_ref = 'Forest_Lan' #this band_names refer to the name of column in shapefile - which column that refer as point reference value
     
    ## Version 2 - Hansen - Sentinel
    
    ## Zone 4: 10 Years rule not OK - > CC overlay with TCL, (Tree Crown Cover) more than the threshold in AOI
    tenYearsRule = minLoss.clip(AOI)
    tenYearsRule_edited = ee.Image(assigning_band(band_name_image,4,tenYearsRule))
    Map.addLayer(tenYearsRule_edited,{'bands': ['Class'],'palette': '#FFA500'},"10 Years Rule - not OK")
    
    
    ## Zone 3: Forest (Sentinel) passed 10 years rule not OK - TCL before year_start_loss overlay with > CC
    # Direct process to get the < year_rule TCL (Explicit)
    #treeLossYear = gfc.select(['lossyear'])
    treeLoss_lessYear = treeLossYear.lt(year_start_loss).selfMask()
    CC_lessYearloss_overlay = AOI_img.And(minArea.And(treeLoss_lessYear))
    # obtain area ten years < CC with the forest masked (Sentinel)
    ForestRegenerating_lessYear = image_y_highbaseline.And(CC_lessYearloss_overlay)
    ForestRegenerating_lessYear_edited = ee.Image(assigning_band(band_name_image,3,ForestRegenerating_lessYear))

    Map.addLayer(ForestRegenerating_lessYear_edited,{'bands': ['Class'],'palette': '#FF0000'},f"Forest (Sentinel) Regenerating - TCL < 20{year_start_loss}")

    ## Zone 5: Water Body Based on Hansen Data
    WaterMask = gfc.select(['datamask']).rename('datamask').eq(2)
    WaterAOI = AOI_img.mask().updateMask(WaterMask)
    # assigning band Class in WaterBody
    water_edited = ee.Image(assigning_band(band_name_image,5,WaterAOI))

    Map.addLayer(water_edited,{'bands': ['Class'],'palette': water},"Water in AOI")
    
    # unmasked the water
    unmaskedWaterAOI = AOI_img.unmask().updateMask(WaterAOI.mask().Not()).clip(AOI)
    
    # Zone 1: Go Zone - No Forest (Sentinel) < start_loss_year in CC
    hiFMasked = image_y_highbaseline.mask()
    maskedHiFI_inverted = hiFMasked.Not()
    unmaskedHiF = AOI_img.unmask().updateMask(maskedHiFI_inverted).clip(AOI)
    GoZoneNoForest_TCL_lessYear = unmaskedHiF.And(CC_lessYearloss_overlay).And(unmaskedWaterAOI)
    GoZoneNoForest_TCL_lessYear_edited = ee.Image(assigning_band(band_name_image,1,GoZoneNoForest_TCL_lessYear))

    Map.addLayer(GoZoneNoForest_TCL_lessYear_edited,{'bands': ['Class'],'palette': '#FFFF00'},"Go Zone - No Forest (Sentinel) in TCL less year >CC")

    ''' #Implying the same as year < year_start_loss and minArea
    ## Zone 3: Forest (Sentinel) 10 years rule not OK - TCL Overlaid with < CC, (Tree Crown Cover) less than the threshold in AOI
    # helper - mask
    tenYearsRule_mask = tenYearsRule.mask()
    masktenYearsRule_inverted = tenYearsRule_mask.Not()
    # Unmasked ten years all - all area outside ten years rule (> CC)
    unmaskedtenYrRuleALL = AOI_img.unmask().updateMask(masktenYearsRule_inverted).clip(AOI)
    # tree loss in the AOI
    tenYearsRuleAll = AOI_img.And(treeLoss)
    # obtain the area in outside ten years rule (>CC) intersected with all ten years rule -> tcl in < CC
    tenYearsRule_notCC = tenYearsRuleAll.And(unmaskedtenYrRuleALL)
    # obtain area ten years < CC with the forest masked (Sentinel)
    Forest10Years_noCC = hiforest_masked.And(tenYearsRule_notCC)
    Forest10Years_noCC_edited = ee.Image(assigning_band(band_name_image,3,Forest10Years_noCC))

    Map.addLayer(Forest10Years_noCC_edited,{'bands': ['Class'],'palette': '#32a844'},"Forest (Sentinel) 10 Years < CC")


    # Zone 1: Go Zone - No Forest (Sentinel) in TCL < CC
    hiFMasked = hiforest_masked.mask()
    maskedHiFI_inverted = hiFMasked.Not()
    unmaskedHiF = AOI_img.unmask().updateMask(maskedHiFI_inverted).clip(AOI)
    GoZoneNoForest_TCLnoCC = unmaskedHiF.And(tenYearsRule_notCC)
    GoZoneNoForest_TCLnoCC_edited = ee.Image(assigning_band(band_name_image,1,GoZoneNoForest_TCLnoCC))

    Map.addLayer(GoZoneNoForest_TCLnoCC_edited,{'bands': ['Class'],'palette': '#FFFF00'},"Go Zone - No Forest (Sentinel) in TCL < CC")
    '''
    
    ## Zone 2: Forest (Hansen > CC year 2000 Baseline) does not overlay with TCL (ALL)
    # helper - unmasked, get area only outside all the ten years rule TCL (ALL)
    # tree loss in the AOI and exclude no loss
    treeLossAll = treeLossYear.gt(0).selfMask()
    #Map.addLayer(treeLossAll,{},'treelossall',False)
    tenYearsRuleAll = AOI_img.And(treeLossAll)
    masked_tenYearsRuleAll = tenYearsRuleAll.mask()
    masked_inverted_tenYearsRuleAll = masked_tenYearsRuleAll.Not()
    unmasked_tenYearsRuleAll = AOI_img.unmask().updateMask(masked_inverted_tenYearsRuleAll).clip(AOI)
    # get the area that is forest in 2000 and has no TCL (No overlay with TCL ALL)
    Forest_RecentHansen = ForestArea2000Hansen.And(unmasked_tenYearsRuleAll)
    # assigning the number Class - 2
    Forest_RecentHansen_edited = ee.Image(assigning_band(band_name_image,2,Forest_RecentHansen))
    Map.addLayer(Forest_RecentHansen_edited,{'bands': ['Class'],'palette': hi_forest},f"Forest_{END_DATE} - Hansen (No TCL in Forest 2000 until {END_DATE}) to be evaluated")

    # Zone 3 - Forest (Sentinel) < CC (ALL TCL)
    # less than threshold
    canopyCover_lessThreshold = canopyCover.lt(cc).selfMask()
    Forest_Regenerating_lessCC = image_y_highbaseline.And(canopyCover_lessThreshold).And(AOI_img)
    Forest_Regenerating_lessCC_edited = ee.Image(assigning_band(band_name_image,3,Forest_Regenerating_lessCC))
    Map.addLayer(Forest_Regenerating_lessCC_edited,{'bands': ['Class'],'palette': '#FF0000'},f"Forest (Sentinel) CC < {tree_cover_forest}% - Regenerating")

    # Zone 1 - Go Zone area (No Forest (Sentinel) and < CC)
    GoZone_NoForestSen_lessThreshold = unmaskedHiF.And(canopyCover_lessThreshold).And(unmaskedWaterAOI)
    GoZone_NoForestSen_lessThreshold_edited = ee.Image(assigning_band(band_name_image,1,GoZone_NoForestSen_lessThreshold))
    Map.addLayer(GoZone_NoForestSen_lessThreshold_edited,{'bands': ['Class'],'palette': '#FFFF00'},"Go Zone - No Forest (Sentinel) < CC")

    if eval_process:
        print(f"EVALUATING ONLY IN FOREST AREA CC > {tree_cover_forest}% THAT GENERATED BY HANSEN 2000 THAT HAS NO TCL")
        ### EVALUATING FOREST CLASS HANSEN
        # CHANGE THE NUMBER Class into 1 as Forest for evaluation purpose only
        Forest_RecentHansen_edited_Class1_Forest = ee.Image(assigning_band(band_name_image,1,Forest_RecentHansen))
        print("Evaluating only Hansen Forest")
        eval = eval_accuracy(band_names_ref, band_name_image,Forest_RecentHansen_edited_Class1_Forest)   
        print("---------------------------- \n")

        ### Evaluating the all end Result Forest Classified (Hansen + Sentinel)

        def get_matrix(image, band_name_image):
            band = image.select([band_name_image])

            # Unmask the band, replacing masked values (null) - Class: null with 0
            unmasked_band = band.unmask(0)

            # Use sampleRegions to extract pixel values at the point locations
            sampled_points = unmasked_band.sampleRegions(
                collection=ref_point_loc,
                scale=10
            )

            pixel_values = sampled_points.aggregate_array(band_name_image).getInfo()

            return pixel_values

        ref_values = gdf_ref[band_names_ref].values

        # prepare for all class into 1, for matrix number assign as forest is 1, no forest is 0, assumed they are not overlapped
        print(f"Evaluating All Forest Classified (Hansen CC > {tree_cover_forest}%) + Sentinel (inGAP) vs Reference value")
        # in the end, if one of them classified forest, then the all result will be class: 1
        ForestRegenerating_lessYear_eval = ee.Image(assigning_band(band_name_image,1,ForestRegenerating_lessYear))   
        Forest_Regenerating_lessCC_eval = ee.Image(assigning_band(band_name_image,1,Forest_Regenerating_lessCC))

        #all_Forest_Hansen_Sentinel = [Forest_RecentHansen_edited_Class1_Forest,ForestRegenerating_lessYear_eval,Forest_Regenerating_lessCC_eval]

        HansenForestOnly_pixValue = get_matrix(Forest_RecentHansen_edited_Class1_Forest, band_name_image)
        print(f"Hansen pix no Loss in CC > {tree_cover_forest}%", HansenForestOnly_pixValue)
        ForestRegenerating_lessYear_pixValue = get_matrix(ForestRegenerating_lessYear_eval, band_name_image)
        print(f"Forest (Sentinel) pix in TCL year < 20{year_start_loss}",ForestRegenerating_lessYear_pixValue)
        Forest_Regenerating_lessCC_pixValue = get_matrix(Forest_Regenerating_lessCC_eval, band_name_image)
        print(f"Forest (Sentinel) pix in CC (year 2000) < {tree_cover_forest}% - Regenerating",Forest_Regenerating_lessCC_pixValue)
        print("------------------------------")

        a = np.array(HansenForestOnly_pixValue)
        b = np.array(ForestRegenerating_lessYear_pixValue)
        c = np.array(Forest_Regenerating_lessCC_pixValue)

        AllForestArr = a + b + c
        AllForestArr_list = AllForestArr.tolist()

        print("Reference Value: ", ref_values)
        print("Pixel Value (All Forest): ",AllForestArr_list)

        confusion_mat_all = confusion_matrix(ref_values, AllForestArr_list)
        print("Confusion matrix summary: \n", confusion_mat_all)

        tn, fp, fn, tp = confusion_matrix(ref_values, AllForestArr_list).ravel()

        print("TN:",tn,"\n FP:",fp, "\n FN:",fn, "\n TP:",tp)

        accuracy = ((tp + tn) / (tp + tn + fp + fn))*100.00
        print("Accuracy: (%) ",accuracy)

        precision = (tp / (tp + fp))*100.00
        print("Precision: (%) ",precision)

        prod_acc = (tp / (tp + fn))*100.00
        print("Producer Accuracy: (%) ",prod_acc) #means that how is the raster result (image) predict forest only,

        user_acc = (tn / (tn + fp))*100.00
        print("User Accuracy: (%) ",user_acc)
        
        
    labels = [f'Forest_{END_DATE} - Hansen (No TCL in Forest 2000 until {END_DATE})', 
          f'Forest (Sentinel) Regenerating - TCL < 20{year_start_loss}', 
          f"Forest (Sentinel) CC < {tree_cover_forest}% - Regenerating",
          "10 Years Rule - not OK",
          'Water', 
          'Go Zone',
         ]
    # colorS can be defined using either hex code or RGB (0-255, 0-255, 0-255)
    colors = [hi_forest, 
              '#FF0000', 
              '#FF0000', 
              '#FFA500', 
               water,
              '#FFFF00',
             ]
    # legend_colors = [(255, 0, 0), (127, 255, 0), (127, 18, 25), (36, 70, 180), (96, 68 123)]

    Map.add_legend(title='Legend', labels=labels, colors=colors)

In [25]:
if small_aoi:
    #create visual boundary color only
    empty = ee.Image().byte()
    AOIm_small = empty.paint(AOIsmall,0,10)

    #Map.addLayer(AOIm,{'palette': '#e043f3'},'AOI') #still buggy
    Map.addLayer(AOIm_small,{},'AOIm_small')
    
if smaller_aoi:
    #create visual boundary color only
    empty = ee.Image().byte()
    AOIm_smaller = empty.paint(AOIsmaller,0,10)

    #Map.addLayer(AOIm,{'palette': '#e043f3'},'AOI') #still buggy
    Map.addLayer(AOIm_smaller,{},'AOIm_smaller')

In [26]:
# quick checking if needed, to uncomment
#water_list = applying_sentinel_water(filtered_scaled, -0.2)
#waterinAOI_ndwi = water_list[0]

#WaterMask = gfc.select(['datamask']).rename('datamask').eq(2)
#WaterAOI = AOI_img.mask().updateMask(WaterMask)
# unmasked the water (hansen)
#unmaskedWaterAOI = AOI_img.unmask().updateMask(WaterAOI.mask().Not()).clip(AOI)

In [27]:
#Map.addLayer(waterinAOI_ndwi,{},'check_ndwi')
#Map.addLayer(unmaskedWaterAOI)

In [28]:
Map.addLayerControl()
Map

Map(center=[-4.887016950796425, 122.69964735710637], controls=(WidgetControl(options=['position', 'transparent…

In [29]:
# Exporting to tiff
if algor_process == 'fcd_galcit' and exporting == True:
    list_images = [tiff_result[0], tiff_result[1], tiff_result[2], tiff_result[3], tiff_result[4]]
    list_names =  ['Go Zone', 'High Forest Baseline', 'High Baseline no pass 10 years rule', 
                           'No pass 10 years rule', 'Water (Un-plantable)']
    for i in range(len(list_images)):
        geemap.ee_export_image_to_drive(
            list_images[i], description=f'{AOI_NAME}_{START_DATE}_{END_DATE}_{list_names[i]}', folder="JupyterGEE", scale=30, region=AOI.geometry(),
        )
        print(f'exported - {list_names[i]}')

In [ ]:
# Define a function to calculate the area of an image within the AOI
def areas_calculation(img):
    # Calculate the total number of pixels in the ROI
    area2 = img.reduceRegion(
      reducer=ee.Reducer.sum(),
      geometry=AOI,
      crs=crs_input,
      scale=30,
      maxPixels=1e13
    )

    print('area calculated \n ------')

    return ee.Number(area2.get('pixel')).getInfo()

In [ ]:
import pandas as pd

if (algor_process == 'Sentinel' or algor_process == 'arief_hansen') and calc_areas:
    # Create a list of images to analyze
    #[goZone_edited,highf_edited,tenyrfl_edited,tenyrule_edited, water_edited]
    list_images = [tiff_result[0], tiff_result[1], tiff_result[2], tiff_result[3], tiff_result[4]]

    # Select the 'pixel' band and calculate the area of each pixel in hectares
    list_areas = [areas_calculation(image.select(['pixel']).multiply(ee.Image.pixelArea()).divide(10000)) for image in list_images]

    # Map the areas_calculation function over the list of images to calculate the area of each image
    #list_areas = ee.List(selected_band_list_images).map(areas_calculation)
    
    
    data = {'Class': [1, 2, 3, 4,5], 
            'Name_Class': ['Go Zone', 'High Forest Baseline', 'High Baseline no pass 10 years rule', 
                           'No pass 10 years rule', 'Water (Un-plantable)'],
            'Area_Ha':list_areas}
    pd.DataFrame.from_dict(data)
    
elif (algor_process == 'Hansen' or algor_process == 'hansen_arief') and calc_areas:
    # Create a list of images to analyze
    #[goZone_edited,highf_edited,tenyrfl_edited,tenyrule_edited, water_edited]
    list_images = [GoZoneNoForest_TCL_lessYear_edited, 
                   GoZone_NoForestSen_lessThreshold_edited, 
                   Forest_RecentHansen_edited, 
                   ForestRegenerating_lessYear_edited,
                   Forest_Regenerating_lessCC_edited, 
                   tenYearsRule_edited,
                   water_edited
                  ]

    # Select the 'pixel' band and calculate the area of each pixel in hectares
    list_areas = [areas_calculation(image.select(['pixel']).multiply(ee.Image.pixelArea()).divide(10000)) for image in list_images]

    # Map the areas_calculation function over the list of images to calculate the area of each image
    #list_areas = ee.List(selected_band_list_images).map(areas_calculation)
    
    data = {'Class': [1,1, 2, 3,3, 4,5], 
        'Name_Class': ['Go Zone - No Forest (Sentinel) in TCL less year >CC', 
                       'Go Zone - No Forest (Sentinel) < CC', 
                       f"Forest_{END_DATE} - Hansen (No TCL in Forest 2000 until {END_DATE}) to be evaluated", 
                       f"Forest (Sentinel) Regenerating - TCL < 20{year_start_loss}", 
                       f"Forest (Sentinel) CC < {tree_cover_forest}% - Regenerating",
                       "10 Years Rule - not OK",
                       'Water (Un-plantable)',
                      ],
        'Area_Ha':list_areas}
    pd.DataFrame.from_dict(data)

In [30]:
print('a')

In [ ]:
#y = e_num**(43.2 - 380 * 0.79 - 0.00970 * 0.28 + 0.916* 0.47 + 6.5* 8.67 - 30.6* 3.94 + 259* 0.5)
#print(y)

In [ ]:
'''#TESTING 

#col_shadow = l9_raw.map(detect_cloud_shadow)

# since if we put inside the cloud_mask_oli to iterate map does not work, hence, we only take the image from one best image based on cloud_cover
best_image_year = l9_raw.filterBounds(AOI) \
                    .filterDate(START_DATE, END_DATE).filter(ee.Filter.lte('CLOUD_COVER', cloud_cover_threshold)) \
                    .sort('CLOUD_COVER', True).first()

#best_image_year = detect_cloud_shadow(best_image_year)
#detect_cloud_shadow(best_image_year)

AlB10 = ee.Number(best_image_year.get('K1_CONSTANT_BAND_10'))
print(AlB10.getInfo())
M1B10 = ee.Number(best_image_year.get('RADIANCE_MULT_BAND_10'))
K_1 = ee.Number(best_image_year.get('K1_CONSTANT_BAND_10'))
K_2 = ee.Number(best_image_year.get('K2_CONSTANT_BAND_10'))
A = ee.Number(best_image_year.get('RADIANCE_ADD_BAND_10'))
M = ee.Number(best_image_year.get('RADIANCE_MULT_BAND_10'))


## TESTING ONE IMAGE FOR TI
temp_image = best_image_year.select(['B10'])
qa = best_image_year.select('QA_PIXEL')
dilated = 1 << 1
cirrus = 1 << 2
cloud = 1 << 3
shadow = 1 << 4
mask = qa.bitwiseAnd(dilated).eq(0) \
    .And(qa.bitwiseAnd(cirrus).eq(0)) \
    .And(qa.bitwiseAnd(cloud).eq(0)) \
    .And(qa.bitwiseAnd(shadow).eq(0))

radiance = temp_image.updateMask(mask).multiply(M1B10).add(AlB10)
        
        #K_1 = ee.Number(image.get('K1_CONSTANT_BAND_10'))
        #K_2 = ee.Number(image.get('K2_CONSTANT_BAND_10'))
        #A = ee.Number(image.get('RADIANCE_ADD_BAND_10'))
        #M = ee.Number(image.get('RADIANCE_MULT_BAND_10'))

        # Getting the Thermal Index - source??
TI = radiance.expression('(K_2) / log((K_1 / TEMP)+1)',{
      'K_2': K_2,
      'K_1': K_1,
      'TEMP':radiance
    }
  ).rename('TI')

TI_norm = normalization_100(TI)                                  
                                  
Map.addLayer(TI_norm,{'min':0,'max':100},'TI_norm_best_images_TESTING')
'''

In [ ]:
a = np.array([0, 1, 0,0])
b = np.array([1, 0, 0,1])
c = np.array([0, 0, 1,1])

result = a + b + c
print(result.tolist())

In [ ]:
##### EXPORTING LANDSAT HANSEN LAST YEAR

AOI_NAME = title_map

geemap.ee_export_image_to_drive(
    LastImageLandsat, description=f'LandsatLast_{END_DATE}_{AOI_NAME}', folder="JupyterGEE", scale=30, region=AOI.geometry(),
)
####################################

In [ ]:
### EXPORTING SENTINEL
geemap.ee_export_image_to_drive(
    s2_sr_median.clip(AOI), description=f'Sentinel_{START_DATE}_{END_DATE}_{AOI_NAME}', folder="JupyterGEE", scale=30, region=AOI.geometry(),
)

In [ ]:
print('a')

In [ ]:
#png_file = './my_map.png'
#Map.to_image(filename=png_file, monitor=1)

html_file = f'./{title_map}.html'
Map.to_html(filename=html_file, title=title_map)

In [ ]:
list_areas

In [ ]:
print(list_areas)

In [ ]:
import pandas as pd
data = {'Class': [1, 2, 3, 4,5], 
        'Name_Class': ['Go Zone', 'High Forest Baseline', 'High Baseline no pass 10 years rule', 
                       'No pass 10 years rule', 'Water (Un-plantable)'],
        'Area_Ha':list_areas}
pd.DataFrame.from_dict(data)

In [ ]:
AOI_NAME = 'SRA_Sentinel'

geemap.ee_export_image_to_drive(
    goZone_edited, description=f'GoZone{AOI_NAME}', folder="JupyterGEE", scale=30, region=AOI.geometry(),
)

geemap.ee_export_image_to_drive(
    highf_edited, description=f'HighBaselineForest{AOI_NAME}', folder="JupyterGEE", scale=30, region=AOI.geometry(),
)

geemap.ee_export_image_to_drive(
    tenyrfl_edited, description=f'HighBaseline10yrsForest{AOI_NAME}', folder="JupyterGEE", scale=30, region=AOI.geometry(),
)

geemap.ee_export_image_to_drive(
    tenyrule_edited, description=f'tenyrsForest{AOI_NAME}', folder="JupyterGEE", scale=30, region=AOI.geometry(),
)

geemap.ee_export_image_to_drive(
    water_edited, description=f'Water{AOI_NAME}', folder="JupyterGEE", scale=30, region=AOI.geometry(),
)

In [ ]:
'''
goZoneArea = goZone.Not().multiply(ee.Image.pixelArea()).divide(10000) #make the zero number into 1 so that we need '.Not()'

# Calculate the total number of pixels in the ROI
area2 = goZoneArea.reduceRegion(
  reducer=ee.Reducer.sum(),
  geometry=AOI,
  crs = 'EPSG:32650',
  scale=30,
  maxPixels=1e13
)
 
goZoneAreaHa = ee.Number(
  area2.get('first')).getInfo()
print(goZoneAreaHa)

highBaselineF_Area = highBaselineF.Not().multiply(ee.Image.pixelArea()).divide(10000) #make the zero number into 1 so that we need '.Not()'

# Calculate the total number of pixels in the ROI
area2_hBF = highBaselineF_Area.reduceRegion(
  reducer=ee.Reducer.sum(),
  geometry=AOI,
  crs = 'EPSG:32650',
  scale=30,
  maxPixels=1e13
)
 
highBaselineF_AreaHa = ee.Number(
  area2_hBF.get('nd')).getInfo()
print(highBaselineF_AreaHa)

waterinAOI_Area = waterinAOI.Not().multiply(ee.Image.pixelArea()).divide(10000) #make the zero number into 1 so that we need '.Not()'

# Calculate the total number of pixels in the ROI
area2_wtr = waterinAOI_Area.reduceRegion(
  reducer=ee.Reducer.sum(),
  geometry=AOI,
  crs = 'EPSG:32650',
  scale=30,
  maxPixels=1e13
)
 
waterinAOI_AreaHa = ee.Number(
  area2_wtr.get('nd')).getInfo()
print(waterinAOI_AreaHa)

HiForestAndLoss_Area = HiForestAndLoss.Not().multiply(ee.Image.pixelArea()).divide(10000) #make the zero number into 1 so that we need '.Not()'

# Calculate the total number of pixels in the ROI
area2_hifl = HiForestAndLoss_Area.reduceRegion(
  reducer=ee.Reducer.sum(),
  geometry=AOI,
  crs = 'EPSG:32650',
  scale=30,
  maxPixels=1e13
)
 
HiForestAndLoss_AreaHa = ee.Number(
  area2_hifl.get('first')).getInfo()
print(HiForestAndLoss_AreaHa)

tenYearsRule_Area = tenYearsRule.Not().multiply(ee.Image.pixelArea()).divide(10000) #make the zero number into 1 so that we need '.Not()'

# Calculate the total number of pixels in the ROI
area2_10yrule = tenYearsRule_Area.reduceRegion(
  reducer=ee.Reducer.sum(),
  geometry=AOI,
  crs = 'EPSG:32650',
  scale=30,
  maxPixels=1e13
)
 
tenYearsRule_AreaHa = ee.Number(
  area2_10yrule.get('first')).getInfo()
print(tenYearsRule_AreaHa)
'''

In [ ]:
'''
zoning_cls = zoning_final.select(['Class'])
zoning_pixel = zoning_final.select(['pixel']).multiply(ee.Image.pixelArea()).divide(10000)

zoning_Area_cls = zoning_pixel.addBands(zoning_cls).reduceRegion(
  reducer=ee.Reducer.sum().group(groupField=1),
  geometry=AOI,
  scale=30,
  maxPixels=1e13
  )

statsFormatted = ee.List(zoning_Area_cls.get('groups')) \
  .map(lambda el: ee.Dictionary(el)) \
  .map(lambda d: [ee.Number(d.get('group')), d.get('sum')])

statsDictionary = ee.Dictionary(statsFormatted.flatten())
print(statsDictionary)
'''

In [ ]:
'''
# Function addition to detect cloud shadows
def detect_cloud_shadow(image):
    l9_not_raw = l9.filterBounds(AOI) \
                    .filterDate(START_DATE, END_DATE).filter(ee.Filter.lte('CLOUD_COVER', cloud_cover_threshold)) \
                    .sort('CLOUD_COVER', True).first()
    
    l9_not_raw_b3 = l9_not_raw.select([('SR_B3')]).multiply(0.0000275).add(-0.2)
    
    shadow_threshold = 0.15

    # Create a mask for potential cloud shadows
    shadow_mask = l9_not_raw_b3.lt(shadow_threshold)
    
    # Apply the shadow mask to the image
    Map.addLayer(shadow_mask,{'min':0, 'max': 1},'shadow_check')
    return image.updateMask(shadow_mask)

# Apply cloud shadow detection function to the image collection
#shadow_detected_collection = collection.map(detect_cloud_shadow)
'''